In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
import json, gensim, sklearn, pickle, sys, re, os
import IPython.display as ipd

In [2]:
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import strip_tags, strip_short, strip_multiple_whitespaces, stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim import utils
from scholarly import scholarly
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from IPython.display import Audio
sound_file = './Music/invalid_keypress.mp3'

In [3]:
datapd=pd.read_json("arxiv-metadata-oai-snapshot.json",  lines=True)

In [4]:
wave = np.sin(2*np.pi*200*np.arange(5000*2)/10000) 
play=Audio(wave, rate=50000, autoplay=True)

## Model 1 -- TF-IDF ## 
### We will first focus on just one subfield -- hep-th ###

In [5]:
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title'] + ['authors']]

In [6]:
datapd[0:5]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [7]:
hep_abstracts[41:45]

,id,abstract,title,authors
444,0704.0445,This paper extends and builds upon the resul...,Geometrically Engineering the Standard Model: ...,Jacob L. Bourjaily
448,0704.0449,We apply mirror symmetry to the problem of c...,"Worldsheet Instantons and Torsion Curves, Part...","Volker Braun, Maximilian Kreuzer, Burt A. Ovru..."
489,0704.0490,The possible existence of axion-like particl...,Long Distance Signaling Using Axion-like Parti...,Daniel D. Stancil
504,0704.0505,We construct a classical solution of an Eins...,Exact Solutions of Einstein-Yang-Mills Theory ...,"Hironobu Kihara, Muneto Nitta"


In [8]:
hep_corpus=hep_abstracts['abstract'].values
processed_hep=preprocess_documents(hep_corpus)

In [9]:
xlen=0
templem=0
stdlen=0
tempstd=0
for i in range(140500):
    templen=len(hep_corpus[i].split())
    i=i+1
    xlen=xlen+templen
meanlen=xlen/140500
print(meanlen)
i=0
for i in range(140500):
    templen=len(hep_corpus[i].split())
    i=i+1
    tempstd=(meanlen-templen)**2
    stdlen=stdlen+tempstd
std=stdlen/140500
print(np.sqrt(std))

114.58131672597865
54.98180209821556


In [10]:
processed_hep[0:5]

[['pure',
  'spinor',
  'formul',
  'dimension',
  'superstr',
  'lead',
  'manifestli',
  'supersymmetr',
  'loop',
  'amplitud',
  'express',
  'integr',
  'pure',
  'spinor',
  'superspac',
  'paper',
  'explor',
  'differ',
  'method',
  'evalu',
  'integr',
  'us',
  'calcul',
  'kinemat',
  'factor',
  'loop',
  'loop',
  'massless',
  'point',
  'amplitud',
  'involv',
  'ramond',
  'state'],
 ['prescript',
  'comput',
  'callia',
  'index',
  'regul',
  'exponenti',
  'function',
  'agreement',
  'old',
  'result',
  'odd',
  'dimens',
  'problem',
  'comput',
  'dimens',
  'moduli',
  'space',
  'self',
  'dual',
  'string',
  'formul',
  'index',
  'problem',
  'dimension',
  'loop',
  'space',
  'think',
  'regul',
  'letter',
  'appli',
  'index',
  'problem'],
 ['studi',
  'space',
  'time',
  'symmetri',
  'scalar',
  'quantum',
  'field',
  'theori',
  'includ',
  'interact',
  'theori',
  'static',
  'space',
  'time',
  'consid',
  'euclidean',
  'quantum',
  'field',


In [11]:
hep_dictionary = gensim.corpora.Dictionary(processed_hep)
hep_bow_corpus = [hep_dictionary.doc2bow(text) for text in processed_hep]

In [12]:
print("Total number of papers in high energy physics theory (hep-th) : ",len(hep_bow_corpus))
print("Size of our dictionary : ", len(hep_dictionary))

Total number of papers in high energy physics theory (hep-th) :  140500
Size of our dictionary :  36210


In [13]:
hep_tfidf = gensim.models.TfidfModel(hep_bow_corpus, smartirs='npu')

In [14]:
index = gensim.similarities.MatrixSimilarity(hep_tfidf[hep_bow_corpus])

In [15]:
def search(string,number):
    new_doc = gensim.parsing.preprocessing.preprocess_string(string)
    new_vec = hep_dictionary.doc2bow(new_doc)
    vec_bow_tfidf = hep_tfidf[new_vec]
    sims = index[vec_bow_tfidf]
    for (s,i) in zip(sorted(enumerate(sims), key=lambda item: -item[1])[:number],range(1,number+1)):
        print( f"{i}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n")

In [16]:
search("ads cft entanglment islands information paradox",15)

1) Quantum Extremal Islands Made Easy, Part I: Entanglement on the Brane by Hong Zhe Chen, Robert C. Myers, Dominik Neuenfeld, Ignacio A. Reyes,
  Joshua Sandor (0.5870394). Arxiv: 2006.04851 

2) Islands outside the horizon by Ahmed Almheiri, Raghu Mahajan, Juan Maldacena (0.58141845). Arxiv: 1910.11077 

3) Islands in Asymptotically Flat 2D Gravity by Thomas Hartman, Edgar Shaghoulian, Andrew Strominger (0.56930244). Arxiv: 2004.13857 

4) Notes on islands in asymptotically flat 2d dilaton black holes by Takanori Anegawa, Norihiro Iizuka (0.54549414). Arxiv: 2004.01601 

5) Islands in cosmology by Thomas Hartman, Yikun Jiang, Edgar Shaghoulian (0.5453892). Arxiv: 2008.01022 

6) Entanglement islands in higher dimensions by Ahmed Almheiri, Raghu Mahajan, and Jorge E. Santos (0.5382712). Arxiv: 1911.09666 

7) Multipartite Purification, Multiboundary Wormholes and Islands in
  AdS$_{3}$/CFT$_{2}$ by Aranya Bhattacharya (0.50951785). Arxiv: 2003.11870 

8) Is the island universe model c

In [17]:
def recommend(string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    parameters={}
    for i in range(len(processed_query3)):
        parameters["sims" + str(i)] = index[vec_test_bow3[i]]
    values=parameters.values()
    for (s,j) in zip(sorted(enumerate(np.sqrt(sum(values))), key=lambda item: -item[1])[:number+len(processed_query3)],range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print( f"{j-len(processed_query3)}) {datapd_hep_th['title'].iloc[s[0]]}.\n Abstracts : {datapd_hep_th['abstract'].iloc[s[0]]}  Authors : {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). \n Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n \n") 

In [18]:
recommend("1005.4690|1107.2116",15)

1) Charged Dilatonic Black Holes and their Transport Properties.
 Abstracts :   We briefly explain the consistency conditions imposed on the effective
holographic theories, which are parameterized by two real exponents
$(\gamma,\delta)$ that control the IR dynamics. The general scaling of DC
resistivity with temperature at low temperature and AC conductivity at low
frequency across the whole $(\gamma,\delta)$ plane are explained. There is a
codimension-one region where the DC resistivity is linear in the temperature.
For massive carriers, it is shown that when the scalar operator is not the
dilaton, the DC resistivity scales as the heat capacity (and entropy) for
$(2+1)$-dimensional systems. Regions are identified where the theory at finite
density is a Mott-like insulator. This contribution is based on arXiv:1005.4690
with emphasis on the transport properties of charged dilatonic black holes with
potential.
  Authors : Blaise Gouteraux, Bom Soo Kim and Rene Meyer (0.9607182). 
 Arxiv:

## Model 2 -- Doc2Vec ##
### Again, just focusing on hep-th ###

In [19]:
def testmodel(model,phrase,number):
    new_doc = gensim.parsing.preprocessing.preprocess_string(phrase)
    test_doc_vector = model.infer_vector(new_doc)
    sims = model.docvecs.most_similar(positive = [test_doc_vector],topn=number)
    for (s,i) in zip(sims,range(1,number+1)):
        print( f"{i}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n") 

In [20]:
def recommend_doc2vec(model,string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    temper=[[] for y in range(len(processed_query3))]
    for i in range(len(processed_query3)):
        temper[i]=model.infer_vector(processed_query3[i])
    sims = model.docvecs.most_similar(positive = temper,topn=1+number+len(processed_query3))
    for (s,j) in zip(sims,range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print( f"{j-len(processed_query3)}) {datapd_hep_th['title'].iloc[s[0]]}.\n Abstracts : {datapd_hep_th['abstract'].iloc[s[0]]}  Authors : {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). \n Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n \n") 

In [21]:
tagged_hep_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_hep)]

In [22]:
model_big_bow300_5_10 = Doc2Vec(tagged_hep_corpus, dm=0, vector_size=300, window=5, min_count=1, epochs=10, hs=1)

In [23]:
recommend_doc2vec(model_big_bow300_5_10,"1212.4103|1212.3616",10)

1) Analytic bootstrap at large spin.
 Abstracts :   We use analytic conformal bootstrap methods to determine the anomalous
dimensions and OPE coefficients for large spin operators in general conformal
field theories in four dimensions containing a scalar operator of conformal
dimension $\Delta_\phi$. It is known that such theories will contain an
infinite sequence of large spin operators with twists approaching
$2\Delta_\phi+2n$ for each integer $n$. By considering the case where such
operators are separated by a twist gap from other operators at large spin, we
analytically determine the $n$, $\Delta_\phi$ dependence of the anomalous
dimensions. We find that for all $n$, the anomalous dimensions are negative for
$\Delta_\phi$ satisfying the unitarity bound. We further compute the first
subleading correction at large spin and show that it becomes universal for
large twist. In the limit when $n$ is large, we find exact agreement with the
AdS/CFT prediction corresponding to the Eikonal li

In [24]:
testmodel(model_big_bow300_5_10,"sparse conformal field theories holgraphy",10)

1) Conformal Field Theory Correlators From sine-Gordon Model on AdS
  Spacetime by Sze-Shiang Feng and Mu-Lin Yan (0.5058285593986511). Arxiv: hep-th/0008179 

2) An Introduction to Conformal Field Theory by Matthias R Gaberdiel (0.48419415950775146). Arxiv: hep-th/9910156 

3) Operator Product Expansions and Consistency Relations in a O(N)
  Invariant Fermionic CFT for 2<d<4 by Anastasios C. Petkou (0.4770517945289612). Arxiv: hep-th/9602054 

4) Holography and the Weyl anomaly by Mans Henningson and Kostas Skenderis (0.4721318185329437). Arxiv: hep-th/9812032 

5) Operator algebra of the SL(2) conformal field theories by Oleg Andreev (0.46654728055000305). Arxiv: hep-th/9504082 

6) On the BCFT Description of Holes in the c=1 Matrix Model by Davide Gaiotto, Nissan Itzhaki and Leonardo Rastelli (0.46646302938461304). Arxiv: hep-th/0307221 

7) Correlation Functions of Local Operators in 2D Gravity Coupled to
  Minimal Matter by Vl.S.Dotsenko (0.46056345105171204). Arxiv: hep-th/911003

In [25]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

# Transfomers-- GPT2, RoBERTa and SBERT #

In [26]:
glove_file = datapath('/home/ubuntu/project/GPT2/abstracts_gpt2_epoch2')
tmp_file = get_tmpfile("/home/ubuntu/project/GPT2/abstracts_gpt2_epoch2_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_gpt2 = KeyedVectors.load_word2vec_format(tmp_file,)

In [27]:
glove_file = datapath('/home/ubuntu/project/Roberta/abstracts_roberta_epoch1_embeddings_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/project/Roberta/abstracts_roberta_epoch1_embeddings_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_roberta = KeyedVectors.load_word2vec_format(tmp_file,)

In [28]:
glove_file = datapath('/home/ubuntu/project/SBERT/abstracts_sbert_mar12_smallersentences_moredata')
tmp_file = get_tmpfile("/home/ubuntu/project/SBERT/abstracts_sbert_mar12_smallersentences_moredata_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_sbert = KeyedVectors.load_word2vec_format(tmp_file,)

In [29]:
tij=hep_abstracts
tij = tij.reset_index(drop=True)
def fastsimplerecommend(string,string1):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    sims = index[vec_test_bow3[0]]
    ind=tij[tij['id'] == string1].index[0]
    if sims[ind]>0.1:
        return (1,sims[ind])
    return (0,0) 

In [30]:
def fast_trans_recommender(model,orig):
    i=1
    j=0
    starter=model.similar_by_word(orig,topn=1000)
    while i<31:
        x=starter[j][0]
        if fastsimplerecommend(orig,x)[0]==1:
            print(f"\n {i}) With TFIDF score {fastsimplerecommend(orig,x)[1]} and GPT-2 rank {j} ")
            print(f" Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {datapd_hep_th.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
            i=i+1
        j=j+1

In [31]:
def plain_recommend(model,string,num):
    for i in range(num):
        x=model.similar_by_word(string,topn=num)[i][0]
        print(f"{i+1}) Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {hep_abstracts.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
        i=i+1
def fast_recommend_lambda(model,string,num):
    for i in range(num):
        x=model.similar_by_word(string,topn=num)[i][0]
        print(f" \"{hep_abstracts.loc[hep_abstracts['id'] == x].id.values[0]}\" , ")
        i=i+1
def fast_lambda_recommender(model,orig):
    i=1
    j=0
    starter=model.similar_by_word(orig,topn=1000)
    while i<31:
        x=starter[j][0]
        if fastsimplerecommend(orig,x)[0]==1:
            print(f"\"{hep_abstracts.loc[hep_abstracts['id'] == x].id.values[0]}\",")
            i=i+1
        j=j+1
def plain_recommend_lambda(string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    parameters={}
    for i in range(len(processed_query3)):
        parameters["sims" + str(i)] = index[vec_test_bow3[i]]
    values=parameters.values()
    for (s,j) in zip(sorted(enumerate(np.sqrt(sum(values))), key=lambda item: -item[1])[:number+len(processed_query3)],range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print(f"\"{datapd_hep_th['id'].iloc[s[0]]}\",")
def plain_recommend_doc2vec(model,string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    temper=[[] for y in range(len(processed_query3))]
    for i in range(len(processed_query3)):
        temper[i]=model.infer_vector(processed_query3[i])
    sims = model.docvecs.most_similar(positive = temper,topn=1+number+len(processed_query3))
    for (s,j) in zip(sims,range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print( f"\"{datapd_hep_th['id'].iloc[s[0]]}\",") 

# Mega testing begins here #

## 1) Jared et al ##

In [33]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1212.3616",30)

"1711.01173",
"1307.6856",
"1502.01437",
"1706.02414",
"1709.00008",
"1912.11100",
"1801.03512",
"1601.05453",
"1710.06419",
"1506.02038",
"1310.5078",
"1309.5089",
"1904.00032",
"1212.4103",
"1305.0004",
"1507.05637",
"1809.09572",
"1702.03938",
"1712.01554",
"2007.07914",
"2009.13137",
"2006.01847",
"hep-th/0002025",
"1404.1094",
"hep-th/9902153",
"1908.00015",
"1611.08407",
"1603.05150",
"1509.00014",
"1612.08471",


In [34]:
plain_recommend_lambda("1212.3616",30)

"1703.06132",
"2007.15668",
"1509.01908",
"1905.09302",
"1711.07343",
"1502.01437",
"2006.05499",
"1811.10646",
"1009.2087",
"0711.1174",
"1703.04744",
"2002.12390",
"1802.04390",
"1801.03512",
"1912.04105",
"1203.5129",
"1607.02195",
"1910.04661",
"1907.08599",
"hep-th/9809096",
"1604.03110",
"1701.04424",
"1711.09075",
"1406.7845",
"2002.12254",
"1710.06419",
"1711.01173",
"1412.8507",
"0807.0004",
"1601.05453",


In [35]:
fast_lambda_recommender(sentence_model_gpt2,"1212.3616")

"0807.0004",
"1403.6829",
"1706.04054",
"1612.08471",
"1307.3784",
"hep-th/9911222",
"1306.5242",
"1906.00667",
"1412.0334",
"1612.03891",
"hep-th/9908160",
"0809.5234",
"1409.3841",
"1203.5129",
"1212.4103",
"2011.02482",
"1612.00885",
"1111.3926",
"1801.04208",
"hep-th/9601023",
"1405.6695",
"1011.5181",
"1810.09406",
"1402.5396",
"1904.07816",
"hep-th/0005182",
"1805.07393",
"1412.7520",
"2007.11647",
"0902.2790",


In [36]:
fast_lambda_recommender(sentence_model_roberta,"1212.3616")

"1710.03199",
"1308.1083",
"2001.03563",
"1306.4662",
"1206.1431",
"1111.3926",
"1907.05147",
"1703.02384",
"1712.01554",
"hep-ph/0605297",
"1708.06772",
"1008.0603",
"hep-th/9903196",
"1910.13578",
"hep-th/9811152",
"1808.03212",
"1011.2986",
"1905.01311",
"1206.1083",
"1712.00161",
"2005.07708",
"0803.3085",
"1711.09075",
"hep-th/0507146",
"1410.4717",
"1711.01173",
"hep-th/0402084",
"1411.7011",
"1704.04500",
"1704.06294",


In [37]:
fast_lambda_recommender(sentence_model_sbert,"1212.3616")

"1609.02165",
"1403.6829",
"2002.12254",
"1709.00008",
"1611.10060",
"1309.5089",
"2002.12390",
"1501.05315",
"1702.00423",
"1912.00222",
"1111.6972",
"1501.02260",
"1305.0004",
"1211.2810",
"1712.07668",
"1608.06241",
"1009.2725",
"1909.05775",
"hep-th/9811152",
"1304.6075",
"1911.12388",
"1612.08471",
"2005.06877",
"1908.06303",
"1904.01876",
"1805.07393",
"1902.05969",
"1601.05453",
"1009.2087",
"1711.09075",


## 2) Lewkowycz Maldacena ##

In [38]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1304.4926",30)

"1805.12145",
"1307.2892",
"1404.3105",
"1403.0951",
"1305.6767",
"1010.3700",
"1303.1884",
"1903.10532",
"1701.01383",
"1309.4523",
"1907.12110",
"1711.08505",
"1912.02799",
"1001.3668",
"1911.11145",
"1305.3448",
"hep-th/0603001",
"1903.10792",
"1905.10121",
"1703.00018",
"1607.07506",
"1905.10084",
"1410.0899",
"1805.10298",
"hep-th/9308077",
"1212.5183",
"2011.00407",
"1811.05382",
"gr-qc/9811071",
"1712.09099",


In [39]:
plain_recommend_lambda("1304.4926",30)

"1404.3105",
"1805.12145",
"1305.6767",
"1307.2892",
"1608.02932",
"1303.1884",
"2011.00407",
"1911.10663",
"1403.0951",
"2005.11338",
"1903.10532",
"hep-th/0606027",
"1601.01694",
"1310.5713",
"1712.09099",
"1601.06788",
"1911.11145",
"1703.04186",
"1505.00826",
"2003.13374",
"1711.08505",
"1311.1685",
"1501.03019",
"1306.4682",
"1711.10854",
"hep-th/0606205",
"1606.04537",
"1701.03113",
"1412.1879",
"1612.04857",


In [40]:
fast_lambda_recommender(sentence_model_gpt2,"1304.4926")

"gr-qc/9406044",
"1505.00826",
"1501.02963",
"2005.09985",
"1607.07506",
"hep-th/0312238",
"hep-th/9802172",
"1201.6367",
"1209.5085",
"1712.09914",
"1404.3105",
"0902.1601",
"cond-mat/0511014",
"0706.3384",
"1110.6008",
"1812.06985",
"hep-th/0402113",
"gr-qc/9911078",
"1505.07842",
"1912.11030",
"0707.1300",
"1711.01276",
"1103.4400",
"gr-qc/0001048",
"1202.2070",
"1602.01473",
"1601.07558",
"1312.3253",
"hep-th/9504033",
"gr-qc/9605073",


In [41]:
fast_lambda_recommender(sentence_model_roberta,"1304.4926")

"1303.1444",
"hep-th/9401125",
"1805.02942",
"2006.16289",
"0911.4708",
"1902.04229",
"1204.6349",
"1202.5273",
"gr-qc/9707015",
"hep-th/0312136",
"1202.5650",
"1309.6935",
"hep-th/9907212",
"1510.00021",
"0903.4707",
"1501.04906",
"1304.7229",
"hep-th/0611093",
"1703.05241",
"gr-qc/0007013",
"1412.1879",
"1211.2649",
"1010.4036",
"1901.06454",
"0912.5118",
"1801.01276",
"1007.1813",
"1105.5925",
"1304.7104",
"hep-th/0010141",


In [42]:
fast_lambda_recommender(sentence_model_sbert,"1304.4926")

"1605.09622",
"1305.6767",
"gr-qc/0607026",
"1904.02170",
"2006.16122",
"1705.08453",
"1507.05897",
"1707.06322",
"1508.04079",
"hep-th/0203219",
"1206.5225",
"1404.6930",
"1605.08588",
"hep-th/0207265",
"1406.5635",
"1509.08486",
"1801.01479",
"1305.2207",
"1806.00450",
"1508.06515",
"1401.5075",
"1602.06756",
"1406.7659",
"hep-th/0606184",
"hep-th/0212029",
"1611.03462",
"1008.4314",
"1103.4620",
"hep-th/0112218",
"0802.1017",


## 3) Maldacena Stanford Yang ##

In [43]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1606.01857",30)

"1809.10672",
"1806.06216",
"hep-th/0604070",
"hep-th/0005052",
"1809.08647",
"1811.01022",
"1111.5623",
"1311.2801",
"hep-th/0012148",
"1410.0487",
"1804.07521",
"1512.02816",
"0807.5016",
"0810.0967",
"1809.02090",
"hep-th/0108050",
"hep-th/9211004",
"hep-th/0601186",
"1606.04108",
"hep-th/9303178",
"1806.03815",
"1604.04099",
"0911.4379",
"hep-th/0402050",
"0705.2149",
"1811.03597",
"1912.07654",
"1203.0674",
"0708.2554",
"hep-th/0307205",


In [44]:
plain_recommend_lambda("1606.01857",30)

"1606.04108",
"1912.07654",
"1809.10672",
"1606.03438",
"1611.10214",
"hep-th/9910076",
"1703.04612",
"hep-th/0006103",
"1811.01022",
"1806.07097",
"1004.1174",
"2009.12370",
"1806.03815",
"1806.06330",
"2011.01953",
"1907.08862",
"1912.12996",
"1308.3863",
"1004.0554",
"2002.00394",
"0812.1576",
"hep-th/0506094",
"0806.2754",
"hep-th/0302132",
"1906.09083",
"1811.04977",
"1005.2255",
"1809.08647",
"1705.04334",
"1009.3139",


In [45]:
fast_lambda_recommender(sentence_model_gpt2,"1606.01857")

"1901.08457",
"1604.07818",
"1611.10214",
"1708.08471",
"1202.0732",
"2011.07071",
"1005.4175",
"1512.09046",
"hep-th/0103162",
"1605.04490",
"2005.08890",
"1505.02986",
"1611.08953",
"1108.4654",
"1912.07654",
"1703.06215",
"1306.3224",
"1301.6524",
"1103.1100",
"1809.08735",
"2009.11147",
"1604.01676",
"1809.07208",
"1605.08697",
"2011.04695",
"0904.0668",
"hep-th/0410034",
"gr-qc/9812052",
"1811.00292",
"1807.01077",


In [46]:
fast_lambda_recommender(sentence_model_roberta,"1606.01857")

"1705.04334",
"1811.00292",
"1802.09547",
"0906.3272",
"1705.09250",
"1903.00478",
"1904.12820",
"0908.0057",
"1706.00771",
"0810.5489",
"1301.6524",
"hep-th/0212096",
"2007.09420",
"1712.02722",
"1511.06842",
"1303.4390",
"1109.0491",
"0812.2234",
"1806.06254",
"1611.03470",
"hep-th/0203219",
"hep-th/0009164",
"0809.0687",
"1810.11442",
"1710.06820",
"1707.05796",
"hep-th/0003081",
"1705.07896",
"1112.3979",
"1501.07499",


In [75]:
fast_trans_recommender(sentence_model_sbert,"1606.01857")


 1) With TFIDF score 0.1644384115934372 and GPT-2 rank 0 
 Paper title : ['Quantum Gravity and the Holographic Principle']. 
 Abstract : ['  In this thesis we study two different approaches to holography, and comment\non the possible relation between them. The first approach is an analysis of the\nhigh-energy regime of quantum gravity in the eikonal approximation, where the\ntheory reduces to a topological field theory. This is the regime where\nparticles interact at high energies but with small momentum transfer. We do\nthis for the cases of asymptotically dS and AdS geometries and find that in\nboth cases the theory is topological. We discuss the relation of our solutions\nin AdS to those of Horowitz and Itzhaki. We also consider quantum gravity away\nfrom the extreme eikonal limit and explain the sense in which the covariance of\nthe theory is equivalent to taking into account transfer of momentum. The\nsecond approach we pursue is the AdS/CFT correspondence. We provide a\nholograp


 7) With TFIDF score 0.06373144686222076 and GPT-2 rank 6 
 Paper title : ['Schr\\"odinger Holography with $z=2$']. 
 Abstract : ['  We investigate holography for asymptotically Schr\\"odinger spacetimes, using\na frame formalism based on the anisotropic scaling symmetry. We build on our\nprevious work on $z<2$ to propose a dictionary for $z=2$. For $z=2$, the\nscaling symmetry does not act on the additional null direction, which implies\nthat in our dictionary it does not correspond to one of the field theory\ndirections. This is significantly different from previous analyses based on\nviewing Schr\\"odinger as a deformation of AdS. We study this dictionary in the\nlinearised theory and in an asymptotic expansion. We show that a solution\nexists in an asymptotic expansion for arbitrary sources for the relevant\noperators in the stress energy complex.\n']. 
 Authors : ['Tomas Andrade, Cynthia Keeler, Alex Peach, Simon F. Ross']. 
 Arxiv ID : ['1412.0031'] 
 
 

 8) With TFIDF score 0.


 14) With TFIDF score 0.10061288625001907 and GPT-2 rank 13 
 Paper title : ['The Graviton in the AdS-CFT correspondence: Solution via the Dirichlet\n  Boundary value problem']. 
 Abstract : ['  Using the AdS-CFT correspondence we calculate the two point function of CFT\nenergy momentum tensors. The AdS gravitons are considered by explicitly solving\nthe Dirichlet boundary value problem for $x_0=\\epsilon$. We consider this\ntreatment as complementary to existing work, with which we make contact.\n']. 
 Authors : ['W. M\\"uck and K. S. Viswanathan']. 
 Arxiv ID : ['hep-th/9810151'] 
 
 

 15) With TFIDF score 0.1272204965353012 and GPT-2 rank 15 
 Paper title : ['Bending AdS Waves with New Massive Gravity']. 
 Abstract : ['  We study AdS-waves in the three-dimensional new theory of massive gravity\nrecently proposed by Bergshoeff, Hohm, and Townsend. The general configuration\nof this type is derived and shown to exhibit different branches, with different\nasymptotic behaviors. In par


 20) With TFIDF score 0.09968562424182892 and GPT-2 rank 21 
 Paper title : ['Back-reaction in massless de Sitter QFTs: holography, gravitational DBI\n  action and f(R) gravity']. 
 Abstract : ['  We employ gauge-gravity duality to study the backreaction effect of\n4-dimensional large-$N$ quantum field theories on constant-curvature\nbackgrounds, and in particular de Sitter space-time. The field theories\nconsidered are holographic QFTs, dual to RG flows between UV and IR CFTs. We\ncompute the holographic QFT contribution to the gravitational effective action\nfor 4d Einstein manifold backgrounds. We find that for a given value of the\ncosmological constant $\\lambda$, there generically exist two backreacted\nconstant-curvature solutions, as long as $\\lambda < \\lambda_{\\textrm{max}} \\sim\nM_p^2 / N^2$, otherwise no such solutions exist. Moreover, the backreaction\neffect interpolates between that of the UV and IR CFTs. We also find that, at\nfinite cutoff, a holographic theory alw


 28) With TFIDF score 0.07728525251150131 and GPT-2 rank 30 
 Paper title : ['New massive gravity, extended']. 
 Abstract : ["  We consider gravity in three dimensions with an arbitrary number of curvature\ncorrections. We show that such corrections are always functions of only three\nindependent curvature invariants. Demanding the existence of a holographic\nc-theorem we show how to fix the coefficients in the action for an arbitrarily\nhigh order, recovering the new massive gravity lagrangian at quadratic order.\nWe calculate the central charge $c$ and show that using Cardy's formula it\nmatches the entropy of black hole solutions, which we construct. We also\nconsider fluctuations about an AdS background, and find that it is possible to\nobtain two derivative equations by imposing a single constraint, thereby\nlifting the pathologic massive modes of new massive gravity. If we do not\nimpose this, there is a set of ghosty massive modes propagating in the bulk.\nHowever, at $c=0$ the

In [47]:
fast_lambda_recommender(sentence_model_sbert,"1606.01857")

"hep-th/0107032",
"2007.08422",
"1608.08781",
"1606.02420",
"1705.04334",
"0911.4257",
"hep-th/9810151",
"0904.0668",
"2003.11417",
"1611.10214",
"hep-th/0204004",
"1611.03470",
"0903.1063",
"1808.05583",
"hep-th/0009164",
"0908.0057",
"2011.04695",
"1411.1948",
"2002.07194",
"1507.05897",
"1706.09080",
"0903.0536",
"2011.01944",
"1306.4937",
"1802.00018",
"1206.1499",
"1801.02589",
"1008.4314",
"1712.02722",
"0809.2912",


## 4) FLM ##

In [48]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1304.4926",30)

"1805.12145",
"1307.2892",
"1404.3105",
"1403.0951",
"1305.6767",
"1701.01383",
"1303.1884",
"1305.3448",
"1903.10532",
"1010.3700",
"1309.4523",
"1711.08505",
"1905.10084",
"1911.11145",
"1001.3668",
"gr-qc/9408040",
"1408.6932",
"1912.02799",
"1905.10121",
"1907.12110",
"1212.5183",
"gr-qc/9811071",
"1211.0832",
"hep-th/9308077",
"hep-th/0603001",
"1410.0899",
"1805.10298",
"2011.00407",
"1908.02044",
"1608.02932",


In [49]:
plain_recommend_lambda("1304.4926",30)

"1404.3105",
"1805.12145",
"1305.6767",
"1307.2892",
"1608.02932",
"1303.1884",
"2011.00407",
"1911.10663",
"1403.0951",
"2005.11338",
"1903.10532",
"hep-th/0606027",
"1601.01694",
"1310.5713",
"1712.09099",
"1601.06788",
"1911.11145",
"1703.04186",
"1505.00826",
"2003.13374",
"1711.08505",
"1311.1685",
"1501.03019",
"1306.4682",
"1711.10854",
"hep-th/0606205",
"1606.04537",
"1701.03113",
"1412.1879",
"1612.04857",


In [50]:
fast_lambda_recommender(sentence_model_gpt2,"1304.4926")

"gr-qc/9406044",
"1505.00826",
"1501.02963",
"2005.09985",
"1607.07506",
"hep-th/0312238",
"hep-th/9802172",
"1201.6367",
"1209.5085",
"1712.09914",
"1404.3105",
"0902.1601",
"cond-mat/0511014",
"0706.3384",
"1110.6008",
"1812.06985",
"hep-th/0402113",
"gr-qc/9911078",
"1505.07842",
"1912.11030",
"0707.1300",
"1711.01276",
"1103.4400",
"gr-qc/0001048",
"1202.2070",
"1602.01473",
"1601.07558",
"1312.3253",
"hep-th/9504033",
"gr-qc/9605073",


In [51]:
fast_lambda_recommender(sentence_model_roberta,"1304.4926")

"1303.1444",
"hep-th/9401125",
"1805.02942",
"2006.16289",
"0911.4708",
"1902.04229",
"1204.6349",
"1202.5273",
"gr-qc/9707015",
"hep-th/0312136",
"1202.5650",
"1309.6935",
"hep-th/9907212",
"1510.00021",
"0903.4707",
"1501.04906",
"1304.7229",
"hep-th/0611093",
"1703.05241",
"gr-qc/0007013",
"1412.1879",
"1211.2649",
"1010.4036",
"1901.06454",
"0912.5118",
"1801.01276",
"1007.1813",
"1105.5925",
"1304.7104",
"hep-th/0010141",


In [52]:
fast_lambda_recommender(sentence_model_sbert,"1304.4926")

"1605.09622",
"1305.6767",
"gr-qc/0607026",
"1904.02170",
"2006.16122",
"1705.08453",
"1507.05897",
"1707.06322",
"1508.04079",
"hep-th/0203219",
"1206.5225",
"1404.6930",
"1605.08588",
"hep-th/0207265",
"1406.5635",
"1509.08486",
"1801.01479",
"1305.2207",
"1806.00450",
"1508.06515",
"1401.5075",
"1602.06756",
"1406.7659",
"hep-th/0606184",
"hep-th/0212029",
"1611.03462",
"1008.4314",
"1103.4620",
"hep-th/0112218",
"0802.1017",


## 5) Maloney Witten  ##

In [53]:
plain_recommend_doc2vec(model_big_bow300_5_10,"0712.0155",30)

"hep-th/0512048",
"1311.2801",
"1610.05814",
"hep-th/9207073",
"0812.0792",
"hep-th/0406130",
"gr-qc/0611028",
"gr-qc/9811071",
"2006.03494",
"hep-th/0412017",
"hep-th/9308011",
"1702.01901",
"0705.2469",
"hep-th/0406043",
"1307.7732",
"1908.03575",
"1308.1108",
"gr-qc/0411065",
"1103.3133",
"gr-qc/9708026",
"hep-th/9603060",
"hep-th/9408068",
"gr-qc/9604051",
"hep-th/9608162",
"1310.2186",
"1501.04252",
"0804.0899",
"hep-th/0302170",
"hep-th/0511140",
"0909.2807",


In [54]:
plain_recommend_lambda("0712.0155",30)

"1304.7021",
"2009.00291",
"hep-th/0202140",
"0707.1159",
"hep-th/0403091",
"hep-th/0506096",
"2006.11503",
"1209.5396",
"1905.01559",
"1310.2186",
"1611.08025",
"hep-th/9810053",
"hep-th/9403077",
"hep-th/9808032",
"1504.00827",
"1111.1161",
"hep-th/0008161",
"1007.0500",
"1511.02004",
"0812.0792",
"1505.04025",
"1903.10510",
"hep-th/9908107",
"hep-th/0412017",
"1611.10194",
"1102.4938",
"1607.06185",
"1503.07418",
"gr-qc/0306054",
"2010.00714",


In [55]:
fast_lambda_recommender(sentence_model_gpt2,"0712.0155")

"0907.0765",
"1712.09914",
"1104.0461",
"2010.08665",
"1201.6367",
"0803.3302",
"hep-th/9612069",
"1512.06206",
"1607.03721",
"1205.5251",
"2004.06605",
"hep-th/9604174",
"1311.6768",
"1405.4192",
"hep-th/9602105",
"1103.5731",
"1806.11134",
"1211.0955",
"1911.02451",
"1308.0027",
"2010.04100",
"1712.00347",
"gr-qc/0101031",
"1208.1254",
"0704.0366",
"1804.01081",
"1410.4590",
"1203.4222",
"1103.1828",
"1210.7747",


In [56]:
fast_lambda_recommender(sentence_model_roberta,"0712.0155")

"hep-th/0210017",
"hep-th/9207095",
"1312.6835",
"hep-th/0001140",
"0711.1879",
"1412.6252",
"0909.1038",
"1404.2190",
"1511.08221",
"1701.05489",
"1712.04465",
"1111.1987",
"hep-th/9607015",
"2004.07419",
"1108.4419",
"hep-th/0602114",
"1808.08962",
"1010.0255",
"1107.0713",
"1701.06629",
"1705.05722",
"hep-th/9612069",
"1212.3344",
"gr-qc/0210086",
"1504.01505",
"hep-th/0412118",
"gr-qc/0406063",
"1206.2630",
"hep-th/9610006",
"gr-qc/9611018",


In [57]:
fast_lambda_recommender(sentence_model_sbert,"0712.0155")

"1508.04079",
"1207.5517",
"1103.4620",
"1507.04757",
"1305.6321",
"1606.06632",
"1105.4733",
"hep-th/0210017",
"hep-th/9811208",
"hep-th/0506176",
"1511.00560",
"hep-th/0503164",
"0808.2627",
"2006.11317",
"hep-th/9705209",
"1909.05253",
"1605.05288",
"1908.03575",
"math/0003082",
"hep-th/0601108",
"1210.0539",
"1705.01953",
"1005.3044",
"0706.3359",
"hep-th/0105171",
"1401.3104",
"hep-th/9710228",
"1811.03597",
"1111.1987",
"0711.1971",


# 6) Slava Ising model #

In [58]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1203.6064",30)

"1009.2725",
"0912.2726",
"1309.5089",
"0807.0004",
"1307.6856",
"1109.6321",
"2006.01847",
"hep-th/9807034",
"1507.05637",
"1911.05747",
"1210.4258",
"1009.2087",
"1708.05718",
"1711.02099",
"1307.3111",
"1601.01310",
"1206.5395",
"1509.00014",
"hep-th/9712074",
"1702.00404",
"1702.03938",
"1711.01173",
"1406.7845",
"1402.6068",
"1504.07997",
"2001.07171",
"1705.04278",
"hep-th/0510067",
"1906.12349",
"hep-th/0610035",


In [59]:
plain_recommend_lambda("1203.6064",30)

"0912.2726",
"0905.2211",
"0807.0004",
"1507.05637",
"1802.04390",
"1506.04659",
"1009.2087",
"1810.07199",
"1109.6321",
"1402.6068",
"hep-th/9611024",
"1210.4258",
"1611.10060",
"1912.04105",
"1903.05306",
"1611.01500",
"1307.3111",
"1711.05530",
"1706.07813",
"2009.07674",
"1907.11247",
"1009.2725",
"1406.7845",
"1403.4545",
"1612.00696",
"2009.12904",
"2002.10147",
"hep-th/0010239",
"1511.01497",
"1811.01913",


In [60]:
fast_lambda_recommender(sentence_model_gpt2,"1203.6064")

"1307.3111",
"1210.4258",
"1810.07199",
"1611.10060",
"1407.7113",
"1211.2810",
"1912.03021",
"1706.01325",
"1409.1581",
"1502.07217",
"1902.05969",
"1605.04175",
"hep-th/0512098",
"1302.6113",
"1601.02883",
"1309.5089",
"1903.05306",
"0705.2834",
"2008.05346",
"hep-th/9510208",
"0911.0043",
"0803.0571",
"1612.07800",
"1707.06224",
"hep-th/0506164",
"1211.6699",
"1504.02822",
"hep-th/9607129",
"cond-mat/0609284",
"2010.07967",


In [61]:
fast_lambda_recommender(sentence_model_roberta,"1203.6064")

"1403.6003",
"hep-th/0702052",
"1512.05362",
"1207.4751",
"1305.1321",
"1810.10015",
"1902.01912",
"0903.2455",
"hep-th/9405123",
"1812.11513",
"2011.03848",
"hep-th/0002218",
"hep-th/9611150",
"gr-qc/0610110",
"0806.3953",
"1711.09105",
"hep-th/0209035",
"hep-th/0208155",
"1709.00008",
"1904.03193",
"0710.0991",
"0705.1983",
"gr-qc/0010031",
"1203.1680",
"1212.6000",
"1203.4894",
"hep-th/0001012",
"1307.5209",
"1112.5620",
"hep-th/0002024",


In [62]:
fast_lambda_recommender(sentence_model_sbert,"1203.6064")

"1712.02314",
"hep-th/9405123",
"1607.06110",
"1902.05969",
"1512.05362",
"1509.00014",
"1906.00707",
"1112.4845",
"1502.07707",
"1611.10060",
"1705.08004",
"1603.05150",
"1911.12388",
"1712.06622",
"1712.07668",
"1705.04278",
"1710.08113",
"1705.03453",
"1703.00278",
"1403.6003",
"1909.05775",
"hep-th/9901144",
"1808.02688",
"1510.08772",
"1210.4258",
"hep-th/9706017",
"1609.00572",
"2002.12390",
"1711.09075",
"1510.08442",


# 7) Juan  SYK # 

In [63]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1604.07818",30)

"1712.02647",
"1810.13217",
"1801.06871",
"1908.00995",
"1707.01740",
"1901.04984",
"1706.05362",
"1608.07567",
"cond-mat/9703073",
"1703.02545",
"1805.05376",
"1901.04561",
"2004.03617",
"2004.05339",
"1706.00412",
"1809.06667",
"1610.01569",
"hep-th/9602054",
"1610.04619",
"1808.01190",
"1701.07081",
"0908.1724",
"1606.02454",
"1802.09547",
"cond-mat/9305042",
"1907.08763",
"1609.07832",
"1603.03020",
"1302.7136",
"1801.09006",


In [64]:
plain_recommend_lambda("1604.07818",30)

"1609.07832",
"1608.07567",
"1906.03308",
"1706.07411",
"1801.08118",
"1609.08631",
"hep-th/0505111",
"1706.00412",
"hep-th/9408018",
"1701.00528",
"1808.01190",
"hep-th/9605079",
"1705.03893",
"1712.02647",
"1902.11086",
"1612.06765",
"1908.00995",
"2007.10352",
"1706.07803",
"2010.10545",
"1912.09975",
"1708.06446",
"1412.7497",
"2002.05725",
"1801.06871",
"1611.10290",
"hep-th/0504146",
"1905.04381",
"1711.09839",
"1610.03816",


In [65]:
fast_lambda_recommender(sentence_model_gpt2,"1604.07818")

"1905.08256",
"1606.03947",
"1610.04394",
"1612.06860",
"1807.04548",
"1708.01160",
"1303.6053",
"1903.00478",
"1910.08459",
"1702.04426",
"1606.01857",
"hep-ph/0303010",
"1606.07477",
"1904.07885",
"1908.00775",
"1612.06330",
"1504.03117",
"1612.06765",
"1909.05847",
"1712.05400",
"1307.0808",
"1810.10626",
"1710.00842",
"1707.02197",
"1503.03839",
"1208.5176",
"1610.03816",
"1606.05239",
"1810.06117",
"cond-mat/0409404",


In [66]:
fast_lambda_recommender(sentence_model_roberta,"1604.07818")

"1004.3664",
"chao-dyn/9803019",
"1703.00831",
"1512.04846",
"1612.00696",
"1610.08917",
"0901.2459",
"1610.09378",
"1009.1615",
"hep-th/9612025",
"1905.08256",
"1912.03124",
"hep-lat/0210013",
"1609.07832",
"1809.00541",
"1907.05354",
"2003.03916",
"hep-ph/0003317",
"2001.11849",
"1510.04983",
"1704.04242",
"2009.08484",
"1109.6666",
"1904.07286",
"1808.00939",
"1906.07728",
"0711.2132",
"1611.03997",
"1612.06765",
"2006.13149",


In [67]:
fast_lambda_recommender(sentence_model_sbert,"1604.07818")

"1811.04330",
"1901.04984",
"1311.3487",
"1610.02422",
"1903.00478",
"2010.07819",
"1810.10626",
"1402.2444",
"1904.03274",
"cond-mat/0407752",
"hep-th/9305031",
"1408.2655",
"cond-mat/9909143",
"1708.06446",
"1609.04501",
"1808.07455",
"1609.07832",
"1404.4263",
"1410.0558",
"1601.06768",
"2002.02066",
"nucl-th/0506085",
"1612.09575",
"hep-lat/9402005",
"1809.06371",
"1508.06668",
"1707.01740",
"1902.02287",
"1510.02520",
"1812.10853",


 # 8) DSD light ray #

In [86]:
plain_recommend_lambda("1805.00098",30)

"1810.05706",
"2010.04726",
"1905.01311",
"1904.05892",
"1311.6957",
"1811.01913",
"2005.06343",
"1904.05905",
"1711.03816",
"1903.10495",
"1408.3566",
"2010.02136",
"2008.07942",
"1803.05086",
"1805.07393",
"1912.08897",
"hep-th/0507162",
"hep-th/0512330",
"0910.5751",
"1002.4480",
"2005.06476",
"hep-th/9901090",
"1608.06284",
"1811.00213",
"1910.12855",
"2007.03743",
"1105.4569",
"hep-th/0507013",
"hep-th/0501116",
"1712.08185",


In [87]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1805.00098",30)

"1810.05706",
"2010.04726",
"1905.01311",
"1910.12855",
"2008.11759",
"1712.08185",
"1904.05892",
"1912.08897",
"2005.06476",
"1810.01439",
"1801.01637",
"1812.09314",
"1711.03816",
"hep-th/9903048",
"1212.3607",
"hep-th/0409258",
"1803.05086",
"1601.05453",
"hep-th/9408009",
"1104.4317",
"2002.00018",
"1703.00278",
"1711.04786",
"1802.04390",
"1610.05308",
"1510.08486",
"1009.2488",
"1812.07044",
"1707.07689",
"1509.02426",


In [84]:
fast_lambda_recommender(sentence_model_sbert,"1805.00098")

"1905.01311",
"1408.3629",
"1907.11247",
"1403.4939",
"1906.03135",
"1910.07424",
"1410.4746",
"2010.04726",
"1311.6957",
"1807.05941",
"1703.00278",
"1611.00763",
"0710.5480",
"1804.09334",
"1906.00707",
"1908.04733",
"1601.05325",
"1711.03816",
"1809.05111",
"1602.08882",
"hep-th/9112021",
"1405.3533",
"1612.07800",
"2007.09395",
"1912.04105",
"hep-th/0004085",
"1502.03404",
"1912.01168",
"1909.00878",
"1410.4185",


In [82]:
fast_lambda_recommender(sentence_model_gpt2,"1805.00098")

"1905.01311",
"1805.07393",
"1705.08004",
"1410.8533",
"1709.03597",
"hep-th/0003080",
"1403.1622",
"1703.00278",
"1903.05306",
"1902.02309",
"hep-th/0409120",
"hep-th/0109064",
"hep-th/0212116",
"1811.02369",
"0806.2531",
"1905.11047",
"hep-th/9403026",
"0806.4794",
"hep-th/0408108",
"1802.10266",
"1604.03110",
"1811.12333",
"2011.02492",
"1702.03938",
"0908.3460",
"1802.05201",
"hep-th/0206020",
"2001.07259",
"1611.02912",
"1810.07199",


In [85]:
fast_lambda_recommender(sentence_model_roberta,"1805.00098")

"1810.07199",
"2001.07259",
"hep-th/0206020",
"2008.03735",
"1709.03597",
"1610.05308",
"1212.3788",
"1108.3077",
"1904.00060",
"1604.03575",
"hep-th/0212223",
"1906.08274",
"hep-th/0106247",
"1611.05577",
"1702.03938",
"hep-th/9201048",
"math-ph/0502001",
"1605.08072",
"1201.5222",
"hep-th/9808120",
"0904.1744",
"2002.12254",
"1904.05905",
"1605.05105",
"1909.05775",
"1110.1063",
"1908.00013",
"2005.03047",
"0801.1561",
"1001.5193",


#  9) Causality Constraints #

In [88]:
fast_lambda_recommender(sentence_model_roberta,"1509.00014")

"1711.05530",
"1912.05561",
"1711.02022",
"1611.10060",
"1609.00572",
"1212.3614",
"1901.10922",
"1506.04983",
"2007.11647",
"1706.10162",
"1911.00522",
"0802.4071",
"1606.07458",
"1708.03182",
"1706.08194",
"1707.01057",
"1703.04589",
"1004.0882",
"hep-th/0205061",
"1803.09902",
"1106.5766",
"1709.10101",
"2008.08553",
"1707.02325",
"1307.7710",
"hep-th/0212340",
"hep-th/0407003",
"gr-qc/9608033",
"hep-th/9408147",
"1804.11330",


In [89]:
fast_lambda_recommender(sentence_model_sbert,"1509.00014")

"2002.10147",
"2005.04234",
"1502.07707",
"1203.6064",
"1912.05580",
"1502.03404",
"1208.0337",
"1701.04830",
"1712.02314",
"1705.04278",
"1811.01913",
"1510.07044",
"1904.00032",
"1602.08570",
"1511.06635",
"1703.00278",
"1702.03938",
"1908.04733",
"1704.03692",
"1607.06110",
"1705.08004",
"1506.04659",
"1712.04861",
"1908.10437",
"1902.05969",
"1610.05308",
"1206.5639",
"1812.05585",
"1402.5698",
"hep-th/0212072",


In [93]:
fast_lambda_recommender(sentence_model_gpt2,"1509.00014")

"1610.05308",
"2005.04234",
"1805.07393",
"1901.10922",
"hep-th/0602178",
"1908.10306",
"1811.00024",
"1605.04175",
"1309.0769",
"1909.05847",
"1204.3894",
"1706.04450",
"0903.1394",
"1802.07033",
"1011.2463",
"1603.03050",
"2010.12818",
"1205.1523",
"1711.05530",
"1606.02606",
"gr-qc/0410017",
"1108.4965",
"1503.08319",
"1410.2534",
"1812.02226",
"1412.7520",
"1910.12123",
"1701.05462",
"1805.11107",
"1609.02429",


In [91]:
plain_recommend_lambda("1509.00014",30)

"1509.06368",
"1606.02771",
"hep-th/0210305",
"1805.11107",
"1406.2335",
"1210.4258",
"1808.08155",
"1803.10233",
"1508.02676",
"1305.0004",
"1904.00032",
"hep-th/9301069",
"2008.12600",
"1607.06109",
"1911.12388",
"2005.04234",
"1705.04278",
"1706.06960",
"1902.05969",
"2005.01721",
"1507.05637",
"1702.08471",
"1611.10060",
"1603.06800",
"1702.05101",
"2006.15859",
"1404.0489",
"1602.07295",
"gr-qc/9705018",
"1601.07904",


In [92]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1509.00014",30)

"2002.12390",
"1808.08155",
"1203.6064",
"1601.07904",
"1910.04661",
"1811.01913",
"1612.00609",
"1802.04390",
"1703.00278",
"1908.06303",
"1307.3745",
"1406.2335",
"1510.03866",
"1208.0337",
"1509.06368",
"hep-th/9607060",
"1404.5300",
"0802.4081",
"1808.03212",
"hep-th/9805162",
"2001.05379",
"hep-th/9410221",
"1306.5242",
"1606.09593",
"2005.09105",
"1812.10462",
"1910.08563",
"1809.09572",
"2001.07171",
"hep-th/0408155",


# 10) ANEC #

In [111]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1605.08072",30) 

"2002.00018",
"1912.08897",
"1703.10656",
"2005.06476",
"1905.06920",
"2006.13317",
"1702.00412",
"1812.04683",
"1802.01585",
"1406.4545",
"1508.00897",
"1608.06292",
"hep-th/9810102",
"hep-th/0304211",
"1511.02288",
"1112.1963",
"hep-th/0109057",
"1604.00965",
"1906.00726",
"1904.08201",
"hep-th/9805145",
"1012.0174",
"hep-th/9904039",
"hep-th/0001154",
"1307.1487",
"1712.04751",
"1705.01486",
"1812.06935",
"1607.06155",
"hep-th/0608158",


In [112]:
plain_recommend_lambda("1605.08072",30)

"2002.00018",
"1705.01486",
"1702.00412",
"1703.10656",
"1809.03516",
"1511.05179",
"1412.5648",
"2002.04637",
"1405.2933",
"1811.05052",
"1812.04683",
"1604.05308",
"1706.09432",
"1406.0520",
"1412.8465",
"1608.06292",
"2006.13317",
"2005.06476",
"1305.3291",
"1512.06431",
"1608.01283",
"1312.7856",
"1611.00016",
"1812.02915",
"1809.10148",
"1508.00766",
"hep-th/0605073",
"1511.02288",
"1703.06523",
"1912.02810",


In [113]:
fast_lambda_recommender(sentence_model_sbert,"1605.08072")

"1407.7816",
"1812.04683",
"1407.8171",
"1304.7100",
"1703.01759",
"1511.06713",
"1904.11495",
"1408.1931",
"1705.08453",
"1812.06985",
"1412.5648",
"1905.02220",
"1203.4007",
"2006.10754",
"1904.12414",
"1709.01804",
"1407.7249",
"1311.1137",
"1909.11402",
"1605.04620",
"1403.1580",
"1804.08899",
"1805.10298",
"2003.09546",
"1309.5453",
"1303.6955",
"1511.05179",
"1006.0047",
"1706.09432",
"1812.08183",


In [114]:
fast_lambda_recommender(sentence_model_gpt2,"1605.08072")

"1207.7119",
"2011.04695",
"1609.04168",
"1412.5648",
"2002.00018",
"1805.07393",
"1810.05151",
"1909.13347",
"2011.01252",
"1607.05656",
"1203.4007",
"2011.09491",
"1909.13808",
"1201.2132",
"1803.10775",
"1407.7816",
"1412.3946",
"1608.08781",
"hep-th/0404007",
"2002.00966",
"1802.00018",
"1412.1037",
"1910.08061",
"1502.03275",
"2001.08237",
"1610.05308",
"1412.7520",
"1810.05338",
"2005.02332",
"1309.0769",


In [115]:
fast_lambda_recommender(sentence_model_roberta,"1605.08072")

"2004.14370",
"hep-th/0106247",
"1608.08781",
"1602.05934",
"1302.0686",
"1802.04278",
"1702.02069",
"hep-th/0406096",
"2008.07994",
"2002.00018",
"hep-th/0004148",
"2003.09898",
"1702.03938",
"1910.08061",
"1701.05276",
"1607.01771",
"1707.02317",
"1702.05497",
"1609.04806",
"1709.07016",
"hep-th/9905104",
"1905.00577",
"1808.02052",
"1706.07056",
"1606.04537",
"1007.4001",
"1010.4396",
"1909.06790",
"hep-th/0610053",
"hep-th/0110133",


# 11) Raghu et al #

In [120]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1908.10996",30) 

"2003.05448",
"2006.11717",
"1912.02210",
"2003.11870",
"1003.0441",
"2007.11658",
"2006.10846",
"2010.12565",
"0704.3719",
"hep-th/0603001",
"2004.07960",
"1404.3105",
"1911.11977",
"2005.08715",
"1405.4817",
"hep-th/0606205",
"2004.01601",
"hep-th/0006126",
"1911.09666",
"hep-th/9404135",
"1309.3610",
"2004.13857",
"2008.05275",
"1911.03402",
"1112.5417",
"hep-th/9312194",
"0903.1667",
"1606.04348",
"1708.00612",
"1306.4004",


In [119]:
plain_recommend_lambda("1908.10996",30)

"2003.05448",
"2007.06800",
"2005.08715",
"2008.05274",
"2011.08814",
"1804.00412",
"1910.00972",
"2006.11717",
"1811.05482",
"1912.08423",
"hep-th/0603081",
"gr-qc/0505068",
"2007.02987",
"1910.12836",
"1512.04538",
"1104.3712",
"1905.08255",
"1507.06410",
"1912.09515",
"gr-qc/0511104",
"2004.01601",
"1401.0590",
"2010.07702",
"0710.2956",
"1612.04857",
"1912.02210",
"1610.08968",
"1311.4329",
"1505.07697",
"2002.03543",


In [116]:
fast_lambda_recommender(sentence_model_sbert,"1908.10996")

"2003.05448",
"1211.6767",
"1808.00867",
"1207.5431",
"1706.02038",
"1402.6378",
"1903.10485",
"1312.3346",
"0705.2070",
"gr-qc/9911091",
"1601.05611",
"1808.05192",
"1312.6887",
"1210.0015",
"1803.04434",
"1810.13440",
"0911.0597",
"1012.4287",
"2004.05863",
"1008.3439",
"hep-th/0207194",
"1910.11077",
"1805.09339",
"1208.3335",
"1305.3479",
"1910.11346",
"1911.06305",
"1412.8600",
"1606.00792",
"0805.1405",


In [117]:
fast_lambda_recommender(sentence_model_gpt2,"1908.10996")

"2003.05448",
"1104.3712",
"1808.03973",
"1406.7304",
"1812.06985",
"1905.08255",
"2007.02987",
"1310.7564",
"1011.5819",
"1611.02647",
"1301.3171",
"gr-qc/9409015",
"0907.0739",
"1808.00867",
"2005.02993",
"1908.06363",
"1006.4090",
"1311.1217",
"1910.00972",
"1702.02069",
"1905.00577",
"1506.05792",
"2008.05275",
"1504.00913",
"1601.07558",
"1808.07966",
"2008.04951",
"1401.2701",
"1712.07529",
"2007.11658",


In [118]:
fast_lambda_recommender(sentence_model_roberta,"1908.10996")

"1910.11077",
"1409.5309",
"1912.00228",
"gr-qc/0405087",
"2006.16289",
"1012.3382",
"gr-qc/0007013",
"1207.0622",
"1507.00354",
"1401.5075",
"gr-qc/0407051",
"1304.2030",
"1304.4926",
"1306.0210",
"0907.2085",
"hep-th/0608213",
"hep-th/9602125",
"2006.10834",
"1708.07243",
"1902.04229",
"1003.1083",
"1502.05949",
"1305.3757",
"1605.02785",
"1208.0198",
"1010.4038",
"1608.00305",
"1506.01018",
"gr-qc/0502025",
"2011.09622",


# 12) Harlow Dong Wall #

In [33]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1601.05416",30) 

"1912.05649",
"hep-th/9905179",
"1408.3203",
"1811.05482",
"1906.05501",
"1703.07780",
"hep-th/0207116",
"1803.10539",
"1912.05262",
"1901.00330",
"1911.07852",
"1904.02170",
"1902.02844",
"1512.06431",
"1911.00519",
"1309.5453",
"2008.07950",
"1802.01040",
"1910.06328",
"0908.3391",
"1903.11244",
"2006.12601",
"hep-th/9210011",
"1705.07943",
"hep-th/9810102",
"0704.3719",
"1812.08183",
"hep-th/9807226",
"1611.02959",
"hep-th/0201100",


In [34]:
plain_recommend_lambda("1601.05416",30)

"1811.05482",
"1607.03605",
"1209.4641",
"1610.08968",
"1411.7041",
"1908.03975",
"1911.00519",
"1912.08423",
"1902.02504",
"1907.08021",
"1605.01070",
"1512.06431",
"1907.06646",
"1910.06329",
"1906.05501",
"1804.05855",
"1807.06041",
"1912.05649",
"1902.02844",
"1705.07943",
"1906.04745",
"1703.07780",
"1204.1698",
"2007.06800",
"1704.05839",
"1803.06680",
"1512.04538",
"1808.00446",
"1808.10169",
"1503.06237",


In [35]:
fast_lambda_recommender(sentence_model_roberta,"1601.05416")

"1503.03542",
"1808.05963",
"hep-th/0401149",
"1402.4497",
"1608.05101",
"0707.3517",
"math-ph/0402072",
"hep-th/0601114",
"1312.6196",
"1705.05678",
"hep-th/0209248",
"1008.3439",
"0805.4234",
"hep-th/0606141",
"1907.07824",
"hep-th/0208002",
"0910.0812",
"1907.01001",
"1505.05069",
"1009.0212",
"hep-th/9401042",
"hep-th/0112166",
"hep-th/0506110",
"1506.01353",
"0903.1063",
"1204.1330",
"hep-th/0105048",
"1609.03560",
"0806.4068",
"1606.07628",


In [36]:
fast_lambda_recommender(sentence_model_sbert,"1601.05416")

"1902.02844",
"1411.7041",
"1704.05839",
"1912.02799",
"1912.05649",
"1410.1540",
"1807.06041",
"1909.06790",
"1710.07643",
"2005.02993",
"1610.08968",
"1705.07943",
"hep-th/0608213",
"2008.04810",
"1611.05841",
"1909.13347",
"1804.05855",
"1902.06905",
"1908.02297",
"1905.06985",
"1811.05171",
"2001.05501",
"1712.04227",
"1905.04317",
"1604.01772",
"1907.06646",
"1910.06329",
"1705.04705",
"1809.09109",
"1203.4007",


In [37]:
fast_lambda_recommender(sentence_model_gpt2,"1601.05416")

"1207.7119",
"1411.7041",
"hep-th/0505028",
"1111.6429",
"1807.06041",
"1607.03901",
"1503.06237",
"1909.13347",
"1910.03190",
"2008.04810",
"1902.02844",
"1709.03597",
"1601.02634",
"1605.08072",
"1906.03669",
"2004.15010",
"0812.2158",
"hep-th/0510046",
"1902.06709",
"1411.7011",
"1801.07280",
"hep-th/0112218",
"1802.10103",
"1106.4501",
"1210.7695",
"1912.11725",
"2007.00855",
"1605.07872",
"2005.02993",
"2010.06106",


# 13) McCough Verlinde #

In [38]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1611.03470",30)

"2011.04664",
"1902.09123",
"1807.11401",
"1810.06110",
"1801.02714",
"1808.07760",
"1405.4519",
"1704.05464",
"1712.09400",
"hep-th/9512128",
"hep-th/0310246",
"hep-th/0203168",
"1411.6356",
"0812.3881",
"1312.6914",
"1701.00341",
"1003.2746",
"1705.10671",
"hep-th/9506067",
"1912.09179",
"0801.2557",
"1412.6668",
"1103.0977",
"1909.02640",
"1007.1148",
"1305.3877",
"1411.1732",
"1104.3281",
"2002.00018",
"1906.10226",


In [39]:
plain_recommend_lambda("1611.03470",30)

"1801.02714",
"1902.01434",
"2004.07242",
"1806.10127",
"1807.11401",
"2005.01693",
"2006.10514",
"1911.05631",
"1911.12359",
"1803.09753",
"1912.09179",
"1909.02640",
"1910.06675",
"1701.05576",
"1907.07221",
"1710.08415",
"1903.07606",
"1711.02716",
"2008.01333",
"1907.03745",
"1911.02031",
"1911.06142",
"1903.06883",
"1906.11251",
"2003.06300",
"1811.00533",
"1808.07760",
"1905.00051",
"1812.00545",
"1907.12603",


In [40]:
fast_lambda_recommender(sentence_model_sbert,"1611.03470")

"1801.02714",
"1907.03497",
"1802.00018",
"1311.7339",
"hep-th/0105048",
"1005.1544",
"1306.3344",
"1408.0415",
"1705.02934",
"1710.08448",
"1811.07758",
"1605.05751",
"1907.01422",
"1808.07760",
"1405.7862",
"1012.5453",
"1602.02039",
"1808.02052",
"1602.08493",
"2003.09435",
"1006.1263",
"1704.06294",
"1503.05538",
"1508.01210",
"1309.0211",
"1906.03894",
"1712.09101",
"1605.02835",
"1711.01027",
"1904.00716",


In [41]:
fast_lambda_recommender(sentence_model_gpt2,"1611.03470")

"2004.07242",
"1608.05534",
"1803.09753",
"1711.02716",
"1911.06142",
"2011.06603",
"1405.4935",
"1906.11251",
"2006.03054",
"1305.1278",
"2006.01835",
"1511.00403",
"2007.04955",
"1708.09376",
"1701.00528",
"1810.04185",
"hep-th/0503192",
"1806.10713",
"1801.07280",
"1908.10372",
"hep-th/0407157",
"1209.5392",
"1808.10306",
"1911.02031",
"1401.1414",
"1307.3784",
"1511.08759",
"0705.3320",
"1408.0795",
"0902.1540",


In [42]:
fast_lambda_recommender(sentence_model_roberta,"1611.03470")

"hep-th/9110056",
"1612.00141",
"1909.11402",
"1811.07965",
"1910.06332",
"1804.10580",
"1110.2513",
"2006.10514",
"1807.08293",
"1411.7011",
"1910.06675",
"1708.08968",
"1607.07418",
"hep-th/0004187",
"2004.12056",
"1910.05342",
"1612.05698",
"1706.00771",
"hep-th/0512167",
"1910.09390",
"0710.3402",
"1012.4678",
"2011.08934",
"1910.13578",
"1201.6452",
"1112.0627",
"hep-th/0203021",
"1902.01434",
"hep-ph/0609254",
"1909.02357",


In [82]:
fast_lambda_recommender(sentence_model_gpt2,"1411.7041")

"1911.03485",
"1510.07911",
"1210.7695",
"1902.02844",
"0803.0571",
"1607.03901",
"1807.06041",
"1601.05416",
"1801.07280",
"1710.01327",
"0911.3160",
"hep-th/0505028",
"1906.03669",
"hep-th/0406225",
"0903.1477",
"2008.04810",
"1503.06237",
"0812.2158",
"2009.04476",
"2010.06106",
"2007.04955",
"2002.03865",
"hep-th/0107032",
"1605.07872",
"hep-th/0406096",
"2004.15010",
"hep-th/0510046",
"1709.04471",
"1503.03542",
"1805.10298",


# 14) Dong et al #

In [86]:
plain_recommend_lambda("1411.7041",30)

"hep-th/9206030",
"1507.07897",
"1503.06237",
"1601.05416",
"hep-th/9311128",
"1102.0478",
"hep-th/9304101",
"1902.02115",
"hep-th/9303018",
"1709.04471",
"1906.05501",
"1904.10812",
"1209.4641",
"1711.01922",
"1905.03268",
"1906.03669",
"0801.2561",
"2008.12869",
"2005.05962",
"1802.01040",
"1911.03485",
"1607.03901",
"1701.00050",
"1609.03560",
"quant-ph/0201018",
"1811.02889",
"2004.04173",
"1902.02844",
"1910.06328",
"1902.02504",


In [85]:
fast_lambda_recommender(sentence_model_roberta,"1411.7041")

"1906.03669",
"2004.15010",
"0907.0151",
"hep-th/0302087",
"0710.2640",
"1502.03404",
"1405.6394",
"1509.03291",
"hep-th/0510046",
"1304.7021",
"hep-th/0505046",
"1511.08025",
"1405.1995",
"1412.5205",
"1306.4974",
"0908.0057",
"1810.02055",
"1910.03190",
"1705.03026",
"hep-th/0606276",
"0911.3160",
"2010.09064",
"1509.01647",
"1503.06237",
"1810.05621",
"2002.04016",
"1611.07528",
"0708.1871",
"1112.1899",
"1611.03470",


In [84]:
fast_lambda_recommender(sentence_model_sbert,"1411.7041")

"1602.04811",
"1601.05416",
"1703.01519",
"1610.08968",
"1912.02799",
"1902.02844",
"1912.05649",
"1607.03605",
"1503.03542",
"1704.05839",
"1908.02297",
"1712.06620",
"1508.00766",
"2008.07022",
"1804.05855",
"1611.05841",
"1703.05328",
"1912.02240",
"1805.08891",
"1909.13347",
"hep-th/0405043",
"1507.07897",
"1311.1137",
"2005.05962",
"1807.06041",
"1910.06329",
"1406.4611",
"1708.01037",
"1703.07780",
"1204.1698",


In [83]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1411.7041",30)

"1811.05171",
"1601.05416",
"1610.02134",
"2005.05962",
"hep-th/9210011",
"1910.06328",
"1701.00050",
"1807.06041",
"1912.02810",
"1408.0415",
"1902.02844",
"1612.04373",
"2006.12601",
"hep-ph/9207202",
"hep-th/0612053",
"1609.03560",
"1905.03787",
"1209.3304",
"hep-th/0405043",
"1503.06237",
"hep-th/9403027",
"1705.01728",
"1507.07897",
"1209.4641",
"1811.05482",
"1503.03542",
"1912.05262",
"1408.3203",
"1404.3105",
"2002.03865",


# 15) Minwalla et al #

In [35]:
plain_recommend_lambda("hep-th/0310285",30)

"hep-th/9908001",
"hep-th/0310286",
"hep-lat/0610076",
"hep-th/0405107",
"hep-th/0402219",
"0803.1880",
"1811.08646",
"hep-th/9810069",
"gr-qc/0102052",
"1806.05729",
"hep-ph/0609254",
"hep-th/0612099",
"1911.06223",
"0706.1768",
"0812.0792",
"1511.05746",
"hep-th/0608151",
"hep-th/0701099",
"1107.4048",
"hep-th/0406210",
"0904.3750",
"1805.03759",
"hep-th/0305179",
"0706.3517",
"1601.01160",
"hep-th/0608115",
"hep-th/0703100",
"0712.3059",
"hep-th/0101134",
"hep-th/0505233",


In [36]:
plain_recommend_doc2vec(model_big_bow300_5_10,"hep-th/0310285",30)

"hep-th/0402219",
"0711.0133",
"1210.5195",
"hep-th/9901076",
"1209.5396",
"1012.5875",
"1007.4052",
"0904.3750",
"hep-th/0211201",
"hep-th/0508092",
"hep-th/0505081",
"2007.13760",
"1907.02524",
"hep-th/0612099",
"hep-th/9810053",
"0801.0305",
"hep-th/0409301",
"0706.3517",
"1104.2592",
"0712.3059",
"1110.3146",
"hep-th/9710080",
"hep-th/9409128",
"hep-th/0101134",
"hep-th/9506002",
"1706.01364",
"hep-th/0512075",
"hep-th/0412017",
"hep-th/0506207",
"1105.2540",


In [43]:
fast_lambda_recommender(sentence_model_sbert,"hep-th/0310285")

"1002.2447",
"1309.5086",
"1003.2694",
"0803.2822",
"1207.3323",
"hep-th/0509002",
"hep-th/9810053",
"1008.1889",
"0911.1392",
"1005.2947",
"1003.1302",
"0911.0075",
"1103.4145",
"hep-th/9908001",
"hep-lat/0610076",
"hep-th/0008155",
"hep-th/0612066",
"hep-th/0602074",
"0808.3953",
"1302.4898",
"1310.5734",
"hep-lat/0202009",
"1201.1331",
"hep-lat/0402021",
"hep-lat/0408004",
"hep-lat/9304015",
"0704.2591",
"cond-mat/0202129",
"hep-th/0302092",
"1209.2532",


In [39]:
fast_lambda_recommender(sentence_model_gpt2,"hep-th/0310285")

"hep-th/0002105",
"1605.07849",
"1909.04592",
"0706.3517",
"1409.8305",
"1309.4394",
"hep-th/0602074",
"hep-lat/0003004",
"hep-th/0407137",
"1005.2928",
"1411.0373",
"2006.16275",
"1805.06463",
"cond-mat/0311222",
"hep-lat/9912003",
"1701.00083",
"1107.4048",
"0911.0075",
"hep-th/0510216",
"1803.11218",
"hep-th/0311177",
"0707.0052",
"gr-qc/0102052",
"1207.3323",
"1211.2843",
"cond-mat/0506529",
"hep-lat/0610076",
"hep-ph/0502255",
"hep-th/0601170",
"1801.01125",


In [42]:
fast_lambda_recommender(sentence_model_roberta,"hep-th/0310285")

"1411.0373",
"1108.0683",
"1212.1238",
"2008.01746",
"cond-mat/0311222",
"hep-ph/0304152",
"1409.8305",
"1107.4048",
"1807.06200",
"1803.11218",
"hep-th/0311177",
"1709.10322",
"1102.5001",
"hep-th/0306104",
"hep-ph/9612282",
"1709.01801",
"1412.3736",
"1111.0580",
"hep-th/0605041",
"1403.0389",
"0904.2444",
"1108.1277",
"hep-th/9608098",
"cond-mat/9808068",
"1109.0502",
"1905.12544",
"2011.09491",
"1703.07322",
"1206.6375",
"2011.08757",


# 16) HRT #

In [44]:
plain_recommend_lambda("0705.0016",30)

"1507.06410",
"hep-th/0603081",
"hep-th/0605073",
"hep-th/0603001",
"1403.1393",
"1511.02288",
"1204.5962",
"1311.4329",
"1612.04857",
"1104.3712",
"1505.07697",
"1305.2728",
"1410.6530",
"1102.0440",
"1302.1878",
"1209.6368",
"0712.1850",
"1903.10532",
"2007.05365",
"1908.02297",
"1911.03841",
"0905.0932",
"1604.07571",
"1412.7170",
"1308.3792",
"1703.04186",
"1303.4442",
"1108.2294",
"1305.3291",
"0710.2956",


In [45]:
fast_lambda_recommender(sentence_model_sbert,"0705.0016")

"2006.10740",
"1601.06788",
"1711.05263",
"1908.02297",
"1305.3448",
"1711.10854",
"1308.3792",
"1905.03777",
"2003.10462",
"1204.1698",
"1507.06410",
"1512.04538",
"1912.02799",
"1510.03876",
"0704.3719",
"1408.6300",
"1406.4611",
"1706.07552",
"1310.4204",
"1404.3105",
"1605.07768",
"1612.04857",
"1901.04554",
"1811.08900",
"1302.1878",
"2009.04990",
"1503.03542",
"1707.05913",
"1607.07506",
"1405.4684",


In [46]:
fast_lambda_recommender(sentence_model_gpt2,"0705.0016")

"2007.02987",
"1908.02297",
"1408.6005",
"1601.01694",
"1508.00766",
"1808.03973",
"1808.07966",
"1504.00913",
"1812.06985",
"1302.0853",
"2006.00799",
"1104.3712",
"1705.09611",
"2010.15737",
"1810.05151",
"1712.07529",
"1203.4007",
"hep-th/0605073",
"hep-th/0608213",
"1705.04705",
"1702.02069",
"1410.1392",
"1811.08879",
"1804.05855",
"1609.03676",
"1807.07357",
"1804.08623",
"1905.00577",
"1302.1878",
"1509.05044",


In [47]:
fast_lambda_recommender(sentence_model_roberta,"0705.0016")

"1702.02069",
"1705.04705",
"1306.4682",
"1811.04085",
"1609.04806",
"1008.3027",
"1709.07016",
"1609.01287",
"1906.04142",
"quant-ph/0611264",
"1605.08072",
"1905.00577",
"1403.2632",
"quant-ph/0211074",
"1601.00136",
"0805.1891",
"1901.08238",
"1302.0853",
"cond-mat/0609072",
"1812.01176",
"1311.1685",
"1804.08623",
"2008.07994",
"1406.2991",
"1807.07055",
"1504.06632",
"1209.3304",
"1808.00867",
"1907.11249",
"1410.1392",


In [50]:
plain_recommend_doc2vec(model_big_bow300_5_10,"0705.0016",30)

"1405.6185",
"1012.4467",
"1811.08900",
"1805.00523",
"2003.13374",
"gr-qc/0212013",
"hep-th/0603081",
"1111.5153",
"1012.1812",
"1611.06220",
"1110.1607",
"1808.05192",
"0905.0932",
"0704.1452",
"1907.12110",
"1610.01542",
"0811.4166",
"1804.07410",
"1102.0440",
"0810.0967",
"1209.4348",
"1205.3113",
"1204.3506",
"hep-th/0008015",
"1609.00250",
"hep-ph/0606143",
"1909.07048",
"1412.6226",
"1908.02560",
"hep-th/9810140",


# 17) HKS #

In [59]:
plain_recommend_lambda("1405.5137",30)

"1101.4937",
"1510.08772",
"1908.02873",
"2004.12557",
"1706.00771",
"1610.06186",
"1805.07393",
"hep-th/0205282",
"hep-th/0010127",
"hep-th/0008140",
"1404.7231",
"1903.06272",
"1711.03122",
"1907.08149",
"hep-th/0107238",
"1010.3561",
"1411.7941",
"hep-th/0104141",
"hep-th/0210272",
"1407.6008",
"1007.0756",
"2003.14316",
"0909.1110",
"hep-th/0105249",
"1911.01448",
"hep-th/0010164",
"1002.1942",
"1706.04612",
"hep-th/0406153",
"1604.07310",


In [52]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1405.5137",30)

"1908.02873",
"1206.5395",
"hep-th/0009244",
"1704.08250",
"2004.12557",
"1903.06272",
"1610.06186",
"1101.4937",
"1106.1897",
"1812.08745",
"1711.03122",
"hep-th/0010127",
"1705.10402",
"0806.0110",
"1904.04833",
"hep-ph/0308290",
"2007.00684",
"1412.0761",
"2009.07313",
"hep-th/9210146",
"1508.00548",
"1404.1094",
"hep-th/9803167",
"0912.2719",
"1212.4103",
"1007.0756",
"1510.02464",
"1602.09089",
"hep-th/0702202",
"1203.0015",


In [55]:
fast_lambda_recommender(sentence_model_gpt2,"1405.5137")

"1802.00018",
"1811.05710",
"1610.05308",
"1308.0329",
"1610.09378",
"1007.1573",
"1103.1427",
"1903.06272",
"1303.7267",
"1912.00222",
"1706.00771",
"1511.02028",
"1412.7520",
"1510.07044",
"2008.02190",
"1805.07393",
"2004.12557",
"1807.04390",
"1906.04184",
"1409.3841",
"hep-lat/9507012",
"1810.01335",
"2005.03676",
"1309.5635",
"hep-th/0409049",
"1305.1278",
"1912.07654",
"1511.07058",
"1705.00292",
"2011.06605",


In [56]:
fast_lambda_recommender(sentence_model_roberta,"1405.5137")

"1812.08757",
"1706.00771",
"2011.06605",
"cond-mat/9407095",
"hep-th/9303030",
"1903.06272",
"2009.10077",
"1801.06118",
"1910.05342",
"1607.05187",
"0712.4212",
"1406.0520",
"hep-th/0508123",
"1412.5205",
"1705.04334",
"1611.10060",
"1904.03532",
"1911.00522",
"hep-th/0003216",
"1603.08524",
"hep-th/9505084",
"1604.07830",
"hep-ph/9901391",
"astro-ph/0212083",
"0803.4087",
"1505.04804",
"1808.03212",
"1607.01784",
"1410.4717",
"1708.08968",


In [57]:
fast_lambda_recommender(sentence_model_sbert,"1405.5137")

"1602.08272",
"1505.04804",
"1804.04151",
"1912.07654",
"1811.12875",
"2010.15813",
"1609.06609",
"1710.08448",
"1812.08745",
"2007.00684",
"1610.09378",
"1705.10402",
"2008.11373",
"1608.00328",
"1810.08015",
"1908.02873",
"1006.1263",
"1706.00771",
"1412.7520",
"1404.5201",
"1509.02160",
"1204.1057",
"1707.07354",
"1603.03771",
"1610.07764",
"1707.07717",
"1312.5510",
"1712.07283",
"2006.13943",
"1711.03122",


# 18) Shenker Saad Cotler et al #

In [60]:
fast_lambda_recommender(sentence_model_sbert,"1611.04650")

"1908.11190",
"1805.01051",
"1109.3571",
"1806.06840",
"1612.04334",
"2006.04623",
"hep-th/0701122",
"1611.04592",
"2005.05673",
"1709.09160",
"1609.03676",
"1702.04350",
"1711.02346",
"1608.04407",
"1705.07597",
"1404.7085",
"1910.10311",
"1712.06963",
"2003.13147",
"1901.06031",
"1312.2839",
"1705.08975",
"1905.02305",
"1211.3425",
"cond-mat/9710040",
"1610.03816",
"1912.05996",
"1910.00123",
"1303.1080",
"1810.09659",


In [61]:
fast_lambda_recommender(sentence_model_gpt2,"1611.04650")

"1612.04334",
"1809.05482",
"1907.01628",
"1702.04350",
"1612.06765",
"1611.04592",
"hep-ph/0003317",
"1612.06330",
"1610.03816",
"astro-ph/0202469",
"1104.4343",
"1809.07228",
"1410.0352",
"1806.06840",
"0909.4947",
"2005.04084",
"gr-qc/0305018",
"1705.08975",
"1612.04792",
"1607.05333",
"hep-th/9705209",
"1706.05400",
"hep-th/9908109",
"1912.11923",
"2008.00039",
"hep-th/0009146",
"1302.0853",
"1512.00169",
"1603.06377",
"1304.7263",


In [62]:
plain_recommend_lambda("1611.04650",30)

"1702.01738",
"1611.04592",
"1612.04334",
"1801.03638",
"1702.06935",
"1710.08184",
"1812.04770",
"1611.10290",
"cond-mat/9407114",
"1706.05400",
"1902.11086",
"1610.03816",
"1612.06330",
"1809.07228",
"1908.10306",
"1911.02026",
"1901.02025",
"0912.4999",
"1702.08902",
"1108.6269",
"1709.06259",
"1209.4295",
"hep-th/0307122",
"1505.08108",
"1812.09469",
"1304.6903",
"1907.04289",
"1912.02835",
"1806.06840",
"1801.06871",


In [64]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1611.04650",30)

"2004.05339",
"1605.06098",
"1610.09758",
"hep-th/0612102",
"1111.2115",
"1601.06768",
"1608.07567",
"0806.3907",
"1008.1059",
"gr-qc/9710020",
"1707.08013",
"1702.01738",
"hep-th/9709114",
"1312.6374",
"2008.12274",
"2002.05725",
"1912.09491",
"hep-th/0412307",
"1812.08785",
"1904.12820",
"math/9903063",
"1709.02810",
"1811.06949",
"1709.06297",
"1703.04612",
"1310.8552",
"hep-th/0610053",
"2001.03158",
"hep-ph/9505364",
"gr-qc/0110076",


In [65]:
fast_lambda_recommender(sentence_model_roberta,"1611.04650")

"1208.0711",
"1706.07439",
"1403.6471",
"hep-th/0203219",
"1904.12820",
"2002.12377",
"1206.2630",
"1712.05073",
"hep-th/0003081",
"1711.05556",
"1207.0819",
"1011.3904",
"hep-th/0412219",
"gr-qc/0311056",
"hep-th/0512167",
"hep-th/0606149",
"0906.3272",
"1112.4664",
"hep-th/0108006",
"1701.05489",
"0912.3520",
"0905.0184",
"hep-th/9506174",
"2003.03914",
"1802.08265",
"gr-qc/0602050",
"1903.11417",
"1411.1744",
"0909.4538",
"1806.11174",


# 19) Caron Huot #

In [66]:
fast_lambda_recommender(sentence_model_sbert,"1703.00278")

"1910.12123",
"hep-th/0611123",
"1906.00707",
"1705.03453",
"1712.02314",
"1812.05585",
"1712.07668",
"1904.00032",
"0911.0043",
"1903.05306",
"1912.00973",
"1612.03891",
"1709.00008",
"1801.04208",
"1708.02017",
"0910.2746",
"1203.6064",
"1905.01311",
"1903.12624",
"1711.02031",
"2005.04234",
"1611.10060",
"1612.08471",
"1910.04661",
"1808.02688",
"1212.4103",
"2005.06440",
"hep-th/0206020",
"1808.03212",
"2006.13964",


In [67]:
fast_lambda_recommender(sentence_model_gpt2,"1703.00278")

"0803.0571",
"1302.0816",
"1802.10266",
"1811.12333",
"1205.5776",
"2006.15813",
"1004.1179",
"1208.4548",
"1308.2276",
"1402.3300",
"2007.04955",
"1607.03901",
"2005.01485",
"1811.02369",
"1810.07199",
"1108.1798",
"1903.12624",
"1205.4412",
"0807.0406",
"0806.1904",
"1303.4752",
"1306.0336",
"2005.04234",
"1908.11316",
"1405.7388",
"hep-th/0602194",
"1802.02524",
"1704.07386",
"1709.03597",
"1007.2412",


In [68]:
plain_recommend_doc2vec(model_big_bow300_5_10,"1703.00278",30)

"1509.00014",
"2006.08221",
"1712.04861",
"1910.04661",
"1805.00098",
"1510.08486",
"1306.5242",
"1910.12855",
"2001.07171",
"2011.03055",
"1302.4420",
"2008.11759",
"1712.02314",
"1703.07361",
"1807.04506",
"1711.03816",
"1904.12883",
"1606.01894",
"1012.2180",
"1311.4886",
"1912.02810",
"1709.00008",
"hep-th/9903048",
"1807.05942",
"1910.02975",
"1802.10266",
"hep-th/0201100",
"1705.02318",
"hep-th/9404158",
"1108.5735",


In [69]:
plain_recommend_lambda("1703.00278",30)

"hep-th/0201100",
"1305.2207",
"hep-th/0208229",
"1802.04390",
"1302.1432",
"1812.02254",
"1510.08091",
"1805.07393",
"2006.08221",
"1510.08772",
"2007.11647",
"1903.05222",
"1912.05580",
"1104.2553",
"1807.05941",
"1504.05754",
"hep-th/0211018",
"1712.08185",
"1712.02314",
"1711.02690",
"1905.12314",
"1711.03816",
"1912.00222",
"0907.2440",
"1201.6300",
"1402.7080",
"1705.03453",
"1802.10266",
"1711.02031",
"1609.01224",


In [70]:
fast_lambda_recommender(sentence_model_roberta,"1703.00278")

"1901.05819",
"1210.7705",
"1111.1463",
"1207.4784",
"1711.04786",
"1806.08181",
"hep-th/0306090",
"1204.0126",
"0707.0424",
"hep-th/9503109",
"hep-th/0111255",
"1402.3208",
"1012.3983",
"hep-th/0404240",
"2006.01922",
"1910.12855",
"1402.6127",
"2005.03071",
"1312.6308",
"hep-th/9911177",
"1001.4018",
"2001.00538",
"1909.01819",
"1011.5482",
"hep-th/9807148",
"1405.3658",
"1503.08838",
"1810.01205",
"1710.06242",
"1508.02554",


# 20) Son #

In [71]:
fast_lambda_recommender(sentence_model_sbert,"hep-th/0205051")

"1407.3415",
"0807.1844",
"1207.3149",
"1012.1280",
"0910.0029",
"1105.0678",
"1007.2653",
"1208.5082",
"1512.07184",
"1511.08025",
"hep-th/0506184",
"1405.0015",
"1001.0779",
"0908.2657",
"1509.01092",
"2005.06169",
"1109.4072",
"1904.00060",
"0712.0802",
"hep-th/0508092",
"0902.0276",
"1004.1179",
"0911.0167",
"1402.0007",
"0903.2596",
"hep-th/9906054",
"1003.5451",
"hep-th/0105131",
"hep-th/0105104",
"1510.05973",


In [72]:
fast_lambda_recommender(sentence_model_gpt2,"hep-th/0205051")

"hep-th/0302026",
"0911.4518",
"0907.5011",
"1302.0536",
"1005.4075",
"0809.0297",
"0809.2789",
"2011.04695",
"0902.1658",
"1012.1280",
"1102.0725",
"1112.4844",
"1001.0779",
"0912.4704",
"0901.4945",
"0706.3215",
"1110.4386",
"1004.1179",
"0708.1871",
"hep-th/0004148",
"hep-th/0205066",
"hep-th/0406274",
"1208.5082",
"1111.6978",
"1201.2132",
"hep-th/0305197",
"1802.00018",
"0908.0057",
"0902.3529",
"hep-th/0205033",


In [74]:
plain_recommend_doc2vec(model_big_bow300_5_10,"hep-th/0205051",30)

"hep-th/0506184",
"1011.5570",
"1312.3815",
"0904.4869",
"0807.1844",
"1409.6357",
"hep-th/0104066",
"hep-th/9807034",
"hep-th/0602010",
"0911.0167",
"hep-th/9907166",
"hep-th/0205052",
"1208.5082",
"hep-th/9903209",
"1203.2636",
"hep-th/0212072",
"1407.3415",
"1006.0157",
"hep-th/9904143",
"hep-th/0603001",
"hep-th/0302026",
"0806.0788",
"0704.0240",
"0903.2209",
"hep-th/0112055",
"1212.6066",
"hep-th/0509079",
"hep-th/0210126",
"hep-th/0207133",
"hep-th/0008179",


In [73]:
plain_recommend_lambda("hep-th/0205051",30)

"hep-th/0302026",
"hep-th/0506184",
"1004.1179",
"1208.5082",
"1312.3815",
"0911.0167",
"1001.0779",
"hep-th/0210220",
"1008.3239",
"0902.1658",
"0903.2209",
"hep-th/9509010",
"0909.1790",
"hep-th/0212072",
"1005.4075",
"hep-th/0612250",
"0910.3722",
"hep-th/9902083",
"1201.1764",
"0902.4010",
"1212.3894",
"hep-th/0112055",
"1803.10793",
"hep-th/0205052",
"0901.3588",
"1307.2511",
"1312.7348",
"hep-ph/0201255",
"hep-th/0609156",
"0903.2596",


# 21) Shiraz fluid gravity #

In [75]:
plain_recommend_lambda("0712.2456",30)

"0811.3468",
"1106.2577",
"0809.2596",
"1012.1040",
"1103.3022",
"1601.01430",
"1408.2173",
"0806.0006",
"0806.4602",
"1210.1561",
"0907.2739",
"1410.0263",
"1401.5975",
"1712.08783",
"1710.04708",
"1201.0681",
"0809.4272",
"1802.08306",
"1410.5843",
"1107.2780",
"0811.2409",
"hep-th/9904079",
"1307.8365",
"1309.6325",
"1207.5309",
"hep-th/0605204",
"2006.10083",
"1206.3982",
"hep-th/0009243",
"1912.01183",


In [76]:
plain_recommend_doc2vec(model_big_bow300_5_10,"0712.2456",30)

"0811.3468",
"1311.3891",
"1105.4530",
"0809.2596",
"0901.3848",
"1012.0174",
"1105.4706",
"hep-ph/0609254",
"0806.4602",
"0809.3505",
"0809.4272",
"1103.3022",
"1307.1367",
"1108.4979",
"1509.08738",
"1109.4072",
"1504.01370",
"1005.2367",
"0706.0678",
"1012.1040",
"0810.4851",
"0811.1665",
"1708.02329",
"1409.3095",
"1406.7222",
"1107.2134",
"1706.03074",
"0810.4354",
"1008.1385",
"1710.04221",


In [77]:
fast_lambda_recommender(sentence_model_gpt2,"0712.2456")

"0806.0006",
"1101.3332",
"1401.5975",
"1709.06766",
"1207.7013",
"1603.01196",
"1103.1814",
"1712.09400",
"1107.5792",
"2002.06801",
"hep-th/0412288",
"hep-th/0401164",
"1612.09241",
"1301.5205",
"1009.1081",
"1808.00847",
"hep-th/0207120",
"1505.05297",
"0708.1187",
"1103.3022",
"0712.2289",
"1111.1161",
"1606.09644",
"hep-th/9712197",
"1210.1561",
"1205.3583",
"1702.04247",
"2009.07874",
"hep-th/0312092",
"hep-th/9411164",


In [78]:
fast_lambda_recommender(sentence_model_sbert,"0712.2456")

"1806.00306",
"0806.0006",
"hep-th/0308109",
"1401.5975",
"hep-th/0308018",
"hep-th/0010141",
"0812.0152",
"1310.0819",
"1509.08738",
"1704.06076",
"0712.2164",
"1105.4530",
"0901.1487",
"1402.6997",
"1309.6325",
"1802.06809",
"0712.2289",
"1504.07616",
"1305.1913",
"1106.2577",
"1807.05058",
"1403.3537",
"0810.2193",
"hep-th/0409307",
"1501.05146",
"1604.05595",
"1304.7085",
"hep-th/0507034",
"hep-th/0612226",
"1211.2815",


In [79]:
fast_lambda_recommender(sentence_model_roberta,"0712.2456")

"2002.10609",
"1902.00854",
"1011.4254",
"1306.1835",
"1009.0175",
"1412.3135",
"0801.3206",
"1510.04566",
"0905.3848",
"hep-th/9307180",
"1107.2780",
"hep-th/0010141",
"1402.6450",
"1803.09495",
"1405.4088",
"1210.4258",
"hep-th/9702150",
"1807.05259",
"1408.2580",
"1205.1573",
"1211.6461",
"1505.04145",
"1511.07371",
"hep-th/9810143",
"1808.00974",
"1312.4953",
"hep-th/9808017",
"1312.0784",
"1402.5204",
"hep-th/9811237",


# Testing different models the human way#

# Verlinde for Edward #

In [50]:
fast_trans_recommender(sentence_model_sbert,"1611.03470")


 1) With TFIDF score 0.4318874478340149 and GPT-2 rank 0 
 Paper title : ['Cutoff AdS$_3$ versus the $T\\bar{T}$ deformation']. 
 Abstract : ['  A recent proposal relates two dimensional holographic conformal field\ntheories deformed by the integrable $T\\bar{T}$ flow to AdS$_3$ with a finite\nradial cutoff. We investigate this proposal by studying perturbative\ncorrelation functions on the two sides. For low point correlators of the stress\ntensor, we successfully match the deformed CFT results at large central charge\nto bulk results obtained in classical pure gravity. The deformed CFT also\nprovides definite predictions for loop corrections in the bulk. We then include\nmatter fields in the bulk. To reproduce the classical bulk two-point function\nof a scalar operator we show that the deformed CFT needs to be augmented with\ndouble trace scalar operators, with the $T\\bar{T}$ operator yielding\ncorrections corresponding to loops in the bulk.\n']. 
 Authors : ['Per Kraus, Junyu Liu,


 9) With TFIDF score 0.14323610067367554 and GPT-2 rank 13 
 Paper title : ['Bulk Phase Shift, CFT Regge Limit and Einstein Gravity']. 
 Abstract : ['  The bulk phase shift, related to a CFT four-point function, describes\ntwo-to-two scattering at fixed impact parameter in the dual AdS spacetime. We\ndescribe its properties for a generic CFT and then focus on large $N$ CFTs with\nclassical bulk duals. We compute the bulk phase shift for vector operators\nusing Regge theory. We use causality and unitarity to put bounds on the bulk\nphase shift. The resulting constraints bound three-point functions of two\nvector operators and the stress tensor in terms of the gap of the theory.\nSimilar bounds should hold for any spinning operator in a CFT. Holographically\nthis implies that in a classical gravitational theory any non-minimal coupling\nto the graviton, as well as any other particle with spin greater than or equal\nto two, is suppressed by the mass of higher spin particles.\n']. 
 Autho


 17) With TFIDF score 0.13610048592090607 and GPT-2 rank 26 
 Paper title : ['Holographic Construction of Excited CFT States']. 
 Abstract : ['  We present a systematic construction of bulk solutions that are dual to CFT\nexcited states. The bulk solution is constructed perturbatively in bulk fields.\nThe linearised solution is universal and depends only on the conformal\ndimension of the primary operator that is associated with the state via the\noperator-state correspondence, while higher order terms depend on detailed\nproperties of the operator, such as its OPE with itself and generally involve\nmany bulk fields. We illustrate the discussion with the holographic\nconstruction of the universal part of the solution for states of two\ndimensional CFTs, either on $R \\times S^1$ or on $R^{1,1}$. We compute the\n1-point function both in the CFT and in the bulk, finding exact agreement. We\ncomment on the relation with other reconstruction approaches.\n']. 
 Authors : ['Ariana Christodo


 25) With TFIDF score 0.12998342514038086 and GPT-2 rank 35 
 Paper title : ['A Note on Holographic Weyl Anomaly and Entanglement Entropy']. 
 Abstract : ["  We develop a general approach to simplify the derivation of the holographic\nWeyl anomaly. As an application, we derive the holographic Weyl anomaly from\ngeneral higher derivative gravity in asymptotically $AdS_{5}$ and $AdS_{7}$.\nInterestingly, to derive all the central charges of 4d and 6d CFTs, we make no\nuse of equations of motion. Following Myers' idea, we propose a formula of\nholographic entanglement entropy for higher derivative gravity in\nasymptotically $AdS_5$. Applying this formula, we obtain the correct universal\nterm of entanglement entropy for 4d CFTs. It turns out that our formula is the\nleading term of Dong's proposal in asymptotically $AdS_5$. Since only the\nleading term contributes to the universal log term, we actually prove that\nDong's proposal yields the correct universal term of entanglement entropy 

In [52]:
recommend_doc2vec(model_big_bow300_5_10,"1611.03470",30) 

1) Geometrizing $T\bar{T}$.
 Abstracts :   The $T\bar{T}$ deformation can be formulated as a dynamical change of
coordinates. We establish and generalize this relation to curved spaces by
coupling the undeformed theory to 2d gravity. For curved space the dynamical
change of coordinates is supplemented by a dynamical Weyl transformation. We
also sharpen the holographic correspondence to cutoff AdS$_3$ in multiple ways.
First, we show that the action of the annular region between the cutoff surface
and the boundary of AdS$_3$ is given precisely by the $T\bar{T}$ operator
integrated over either the cutoff surface or the asymptotic boundary. Then we
derive dynamical coordinate and Weyl transformations directly from the bulk.
Finally, we reproduce the flow equation for the deformed stress tensor from the
cutoff geometry.
  Authors : Pawel Caputa, Shouvik Datta, Yunfeng Jiang, Per Kraus (0.3641091585159302). 
 Arxiv: 2011.04664 
 

2) Scrambling time from local perturbations of the rotating 

In [51]:
recommend("1611.03470",30)

1) Cutoff AdS$_3$ versus the $T\bar{T}$ deformation.
 Abstracts :   A recent proposal relates two dimensional holographic conformal field
theories deformed by the integrable $T\bar{T}$ flow to AdS$_3$ with a finite
radial cutoff. We investigate this proposal by studying perturbative
correlation functions on the two sides. For low point correlators of the stress
tensor, we successfully match the deformed CFT results at large central charge
to bulk results obtained in classical pure gravity. The deformed CFT also
provides definite predictions for loop corrections in the bulk. We then include
matter fields in the bulk. To reproduce the classical bulk two-point function
of a scalar operator we show that the deformed CFT needs to be augmented with
double trace scalar operators, with the $T\bar{T}$ operator yielding
corrections corresponding to loops in the bulk.
  Authors : Per Kraus, Junyu Liu, Donald Marolf (0.65718144). 
 Arxiv: 1801.02714 
 

2) On correlation functions in $J\bar T$-def

In [53]:
fast_trans_recommender(sentence_model_gpt2,"1611.03470")


 1) With TFIDF score 0.390150785446167 and GPT-2 rank 0 
 Paper title : ['JT gravity at finite cutoff']. 
 Abstract : ['  We compute the partition function of $2D$ Jackiw-Teitelboim (JT) gravity at\nfinite cutoff in two ways: (i) via an exact evaluation of the Wheeler-DeWitt\nwave-functional in radial quantization and (ii) through a direct computation of\nthe Euclidean path integral. Both methods deal with Dirichlet boundary\nconditions for the metric and the dilaton. In the first approach, the radial\nwavefunctionals are found by reducing the constraint equations to two first\norder functional derivative equations that can be solved exactly, including\nfactor ordering. In the second approach we perform the path integral exactly\nwhen summing over surfaces with disk topology, to all orders in perturbation\ntheory in the cutoff. Both results precisely match the recently derived\npartition function in the Schwarzian theory deformed by an operator analogous\nto the $T\\bar{T}$ deformatio


 7) With TFIDF score 0.1827225387096405 and GPT-2 rank 9 
 Paper title : ['Thermodynamics of a BTZ black hole solution with an Horndeski source']. 
 Abstract : ['  In three dimensions, we consider a particular truncation of the Horndeski\naction that reduces to the Einstein-Hilbert Lagrangian with a cosmological\nconstant $\\Lambda$ and a scalar field whose dynamics is governed by its usual\nkinetic term together with a nonminimal kinetic coupling. Requiring the radial\ncomponent of the conserved current to vanish, the solution turns out to be the\nBTZ black hole geometry with a radial scalar field well-defined at the horizon.\nThis means in particular that the stress tensor associated to the matter source\nbehaves on-shell as an effective cosmological constant term. We construct an\nEuclidean action whose field equations are consistent with the original ones\nand such that the constraint on the radial component of the conserved current\nalso appears as a field equation. With the help


 13) With TFIDF score 0.13449305295944214 and GPT-2 rank 20 
 Paper title : ['CFT in AdS and boundary RG flows']. 
 Abstract : ['  Using the fact that flat space with a boundary is related by a Weyl\ntransformation to anti-de Sitter (AdS) space, one may study observables in\nboundary conformal field theory (BCFT) by placing a CFT in AdS. In addition to\ncorrelation functions of local operators, a quantity of interest is the free\nenergy of the CFT computed on the AdS space with hyperbolic ball metric, i.e.\nwith a spherical boundary. It is natural to expect that the AdS free energy can\nbe used to define a quantity that decreases under boundary renormalization\ngroup (RG) flows. We test this idea by discussing in detail the case of the\nlarge $N$ critical $O(N)$ model in general dimension $d$, as well as its\nperturbative descriptions in the epsilon-expansion. Using the AdS approach, we\nrecover the various known boundary critical behaviors of the model, and we\ncompute the free energ


 19) With TFIDF score 0.12016238272190094 and GPT-2 rank 40 
 Paper title : ['Bulk metric reconstruction from boundary entanglement']. 
 Abstract : ['  Most of the literature in the \\emph{bulk reconstruction program} in\nholography focuses on recovering local bulk operators propagating on a\nquasilocal bulk geometry and the knowledge of the bulk geometry is always\nassumed or guessed. The fundamental problem of the bulk reconstruction program,\nwhich is \\emph{recovering the bulk background geometry (metric)} from the\nboundary CFT state is still outstanding. In this work, we formulate a recipe to\nextract the bulk metric itself from the boundary state, specifically, the\nmodular Hamiltonian information of spherical subregions in the boundary. Our\nrecipe exploits the recent construction of Kabat and Lifschytz\n\\cite{Kabat:2017mun} to first compute the bulk two point function of scalar\nfields directly in the CFT without knowledge of the bulk metric or the\nequations of motion, and 


 25) With TFIDF score 0.19602015614509583 and GPT-2 rank 56 
 Paper title : ['Recent progress in the effective string theory description of LGTs']. 
 Abstract : ['  In presence of a static pair of sources, the spectrum of low-lying states of\nany confining gauge theory in D space-time dimensions is described, at large\nsource separations, by an effective string theory. Recently two important\nadvances improved our understanding of this effective theory. First, it was\nrealized that the form of the effective action is strongly constrained by the\nrequirement of the Lorentz invariance of the gauge theory, which is\nspontaneously broken by the formation of a long confining flux tube in the\nvacuum. This constraint is strong enough to fix uniquely the first few\nsubleading terms of the action. Second, it has been realized that the first of\nthese allowed terms - a quartic polynomial in the field derivatives - is\nexactly the composite field $T\\bar{T}$, built with the chiral components, $

# Dong for Edward #

In [87]:
fast_trans_recommender(sentence_model_gpt2,"1411.7041")


 1) With TFIDF score 0.3396114706993103 and GPT-2 rank 0 
 Paper title : ['Central charges of aperiodic holographic tensor network models']. 
 Abstract : ['  Central to the AdS/CFT correspondence is a precise relationship between the\ncurvature of an anti-de Sitter (AdS) spacetime and the central charge of the\ndual conformal field theory (CFT) on its boundary. Our work shows that such a\nrelationship can also be established for tensor network models of AdS/CFT based\non regular bulk geometries, leading to an analytical form of the maximal\ncentral charges exhibited by the boundary states. We identify a class of\ntensors based on Majorana dimer states that saturate these bounds in the large\ncurvature limit, while also realizing perfect and block-perfect holographic\nquantum error correcting codes. Furthermore, the renormalization group\ndescription of the resulting model is shown to be analogous to the strong\ndisorder renormalization group, thus giving the first example of an exact\


 7) With TFIDF score 0.30242323875427246 and GPT-2 rank 6 
 Paper title : ['Learning the Alpha-bits of Black Holes']. 
 Abstract : ["  When the bulk geometry in AdS/CFT contains a black hole, the boundary\nreconstruction of a given bulk operator will often necessarily depend on the\nchoice of black hole microstate, an example of state dependence. As a result,\nwhether a given bulk operator can be reconstructed on the boundary at all can\ndepend on whether the black hole is described by a pure state or thermal\nensemble. We refine this dichotomy, demonstrating that the same boundary\noperator can often be used for large subspaces of black hole microstates,\ncorresponding to a constant fraction $\\alpha$ of the black hole entropy. In the\nSchrodinger picture, the boundary subregion encodes the $\\alpha$-bits (a\nconcept from quantum information) of a bulk region containing the black hole\nand bounded by extremal surfaces. These results have important consequences for\nthe structure of A


 13) With TFIDF score 0.35220402479171753 and GPT-2 rank 21 
 Paper title : ['Eigenstate Thermalization Hypothesis and Approximate Quantum Error\n  Correction']. 
 Abstract : ['  The eigenstate thermalization hypothesis (ETH) is a powerful conjecture for\nunderstanding how statistical mechanics emerges in a large class of many-body\nquantum systems. It has also been interpreted in a CFT context, and, in\nparticular, holographic CFTs are expected to satisfy ETH. Recently, it was\nobserved that the ETH condition corresponds to a necessary and sufficient\ncondition for an approximate quantum error correcting code (AQECC), implying\nthe presence of AQECCs in systems satisfying ETH. In this paper, we explore the\nproperties of ETH as an error correcting code and show that there exists an\nexplicit universal recovery channel for the code. Based on the analysis, we\ndiscuss a generalization that all chaotic theories contain error correcting\ncodes. We then specialize to AdS/CFT to demonstrat


 19) With TFIDF score 0.16163693368434906 and GPT-2 rank 34 
 Paper title : ["Inside the Hologram: Reconstructing the bulk observer's experience"]. 
 Abstract : ['  We develop a holographic framework for describing the experience of bulk\nobservers in AdS/CFT, that allows us to compute the proper time and energy\ndistribution measured along any bulk worldline. Our method is formulated\ndirectly in the CFT language and is universal: It does not require knowledge of\nthe bulk geometry as an input. When used to propagate operators along the\nworldline of an observer falling into an eternal black hole, our proposal\nresolves a conceptual puzzle raised by Marolf and Wall. Notably, the\nprescription does not rely on an external dynamical Hamiltonian or the AdS\nboundary conditions and is, therefore, outlining a general framework for the\nemergence of time.\n']. 
 Authors : ['Daniel Louis Jafferis and Lampros Lamprou']. 
 Arxiv ID : ['2009.04476'] 
 
 

 20) With TFIDF score 0.17230007052421


 25) With TFIDF score 0.13921543955802917 and GPT-2 rank 49 
 Paper title : ['Giant gravitons in AdS/CFT (I): matrix model and back reaction']. 
 Abstract : ['  In this article we study giant gravitons in the framework of AdS/CFT\ncorrespondence. First, we show how to describe these configurations in the CFT\nside using a matrix model. In this picture, giant gravitons are realized as\nsingle excitations high above a Fermi sea, or as deep holes into it. Then, we\ngive a prescription to define quasi-classical states and we recover the known\nclassical solution associated to the CFT dual of a giant graviton that grows in\nAdS. Second, we use the AdS/CFT dictionary to obtain the supergravity boundary\nstress tensor of a general state and to holographically reconstruct the bulk\nmetric, obtaining the back reaction of space-time. We find that the space-time\nresponse to all the supersymmetric giant graviton states is of the same form,\nproducing the singular BPS limit of the three charge Re

In [88]:
recommend_doc2vec(model_big_bow300_5_10,"1411.7041",30) 

1) Holographic Renyi Entropy from Quantum Error Correction.
 Abstracts :   We study Renyi entropies $S_n$ in quantum error correcting codes and compare
the answer to the cosmic brane prescription for computing $\widetilde{S}_n
\equiv n^2 \partial_n (\frac{n-1}{n} S_n)$. We find that general operator
algebra codes have a similar, more general prescription. Notably, for the
AdS/CFT code to match the specific cosmic brane prescription, the code must
have maximal entanglement within eigenspaces of the area operator. This gives
us an improved definition of the area operator, and establishes a stronger
connection between the Ryu-Takayanagi area term and the edge modes in lattice
gauge theory. We also propose a new interpretation of existing holographic
tensor networks as area eigenstates instead of smooth geometries. This
interpretation would explain why tensor networks have historically had trouble
modeling the Renyi entropy spectrum of holographic CFTs, and it suggests a
method to construc

# Harlow for Ronak #

In [43]:
fast_trans_recommender(sentence_model_sbert,"1601.05416")


 1) With TFIDF score 0.33542338013648987 and GPT-2 rank 0 
 Paper title : ['Entanglement Wedge Reconstruction using the Petz Map']. 
 Abstract : ['  At the heart of recent progress in AdS/CFT is the question of subregion\nduality, or entanglement wedge reconstruction: which part(s) of the boundary\nCFT are dual to a given subregion of the bulk? This question can be answered by\nappealing to the quantum error correcting properties of holography, and it was\nrecently shown that robust bulk (entanglement wedge) reconstruction can be\nachieved using a universal recovery channel known as the twirled Petz map. In\nshort, one can use the twirled Petz map to recover bulk data from a subset of\nthe boundary. However, this map involves an averaging procedure over bulk and\nboundary modular time, and hence it can be somewhat intractable to evaluate in\npractice. We show that a much simpler channel, the Petz map, is sufficient for\nentanglement wedge reconstruction for any code space of fixed fin


 7) With TFIDF score 0.3377576470375061 and GPT-2 rank 6 
 Paper title : ['Learning the Alpha-bits of Black Holes']. 
 Abstract : ["  When the bulk geometry in AdS/CFT contains a black hole, the boundary\nreconstruction of a given bulk operator will often necessarily depend on the\nchoice of black hole microstate, an example of state dependence. As a result,\nwhether a given bulk operator can be reconstructed on the boundary at all can\ndepend on whether the black hole is described by a pure state or thermal\nensemble. We refine this dichotomy, demonstrating that the same boundary\noperator can often be used for large subspaces of black hole microstates,\ncorresponding to a constant fraction $\\alpha$ of the black hole entropy. In the\nSchrodinger picture, the boundary subregion encodes the $\\alpha$-bits (a\nconcept from quantum information) of a bulk region containing the black hole\nand bounded by extremal surfaces. These results have important consequences for\nthe structure of Ad


 13) With TFIDF score 0.1127946749329567 and GPT-2 rank 13 
 Paper title : ['AdS/CFT and Strong Subadditivity of Entanglement Entropy']. 
 Abstract : ['  Recently, a holographic computation of the entanglement entropy in conformal\nfield theories has been proposed via the AdS/CFT correspondence. One of the\nmost important properties of the entanglement entropy is known as the strong\nsubadditivity. This requires that the entanglement entropy should be a concave\nfunction with respect to geometric parameters. It is a non-trivial check on the\nproposal to see if this property is indeed satisfied by the entropy computed\nholographically. In this paper we examine several examples which are defined by\nannuli or cusps, and confirm the strong subadditivity via direct calculations.\nFurthermore, we conjecture that Wilson loop correlators in strongly coupled\ngauge theories satisfy the same relation. We also discuss the relation between\nthe holographic entanglement entropy and the Bousso bou


 19) With TFIDF score 0.23421573638916016 and GPT-2 rank 21 
 Paper title : ['Holographic entanglement entropy is cutoff-covariant']. 
 Abstract : ['  In the AdS/CFT correspondence, it is often convenient to regulate infinite\nquantities in asymptotically anti-de Sitter spacetimes by introducing a sharp\ncutoff in a radial coordinate. This procedure is a priori coordinate-dependent,\nand may not be well-motivated in full, covariant general relativity; however,\nthe fact that physically meaningful quantities such as the entanglement entropy\ncan be obtained by such a regulation procedure suggests some underlying\ncovariance. In this paper, we provide a careful treatment of the radial cutoff\nprocedure for computing holographic entanglement entropy in asymptotically\nanti-de Sitter spacetimes. We prove two results that are frequently assumed in\nthe literature, but that have not been carefully addressed: (i) that the choice\nof a "globally minimal surface" among several extremal candida


 24) With TFIDF score 0.11726105213165283 and GPT-2 rank 28 
 Paper title : ['Towards a Bit Threads Derivation of Holographic Entanglement of\n  Purification']. 
 Abstract : ['  We apply the bit thread formulation of holographic entanglement entropy to\nreduced states describing only the geometry contained within an entanglement\nwedge. We argue that a certain optimized bit thread configuration, which we\nconstruct, gives a purification of the reduced state to a full holographic\nstate obeying a precise set of conditional mutual information relations. When\nthis purification exists, we establish, under certain assumptions, the\nconjectured $E_P = E_W$ relation equating the entanglement of purification with\nthe area of the minimal cross section partitioning the bulk entanglement wedge.\nAlong the way, we comment on minimal purifications of holographic states,\ngeometric purifications, and black hole geometries.\n']. 
 Authors : ['Ning Bao, Aidan Chatwin-Davies, Jason Pollack, Grant N.

In [45]:
recommend_doc2vec(model_big_bow300_5_10,"1601.05416",30) 

1) Quantum Extremal Surfaces: Holographic Entanglement Entropy beyond the
  Classical Regime.
 Abstracts :   We propose that holographic entanglement entropy can be calculated at
arbitrary orders in the bulk Planck constant using the concept of a "quantum
extremal surface": a surface which extremizes the generalized entropy, i.e. the
sum of area and bulk entanglement entropy. At leading order in bulk quantum
corrections, our proposal agrees with the formula of Faulkner, Lewkowycz, and
Maldacena, which was derived only at this order; beyond leading order
corrections, the two conjectures diverge. Quantum extremal surfaces lie outside
the causal domain of influence of the boundary region as well as its
complement, and in some spacetimes there are barriers preventing them from
entering certain regions. We comment on the implications for bulk
reconstruction.
  Authors : Netta Engelhardt and Aron C. Wall (0.426145076751709). 
 Arxiv: 1408.3203 
 

2) Holographic Relative Entropy in Infinite-

In [49]:
recommend("1601.05416",30)

1) Holographic Relative Entropy in Infinite-dimensional Hilbert Spaces.
 Abstracts :   We reformulate entanglement wedge reconstruction in the language of
operator-algebra quantum error correction with infinite-dimensional physical
and code Hilbert spaces. Von Neumann algebras are used to characterize
observables in a boundary subregion and its entanglement wedge. Assuming that
the infinite-dimensional von Neumann algebras associated with an entanglement
wedge and its complement may both be reconstructed in their corresponding
boundary subregions, we prove that the relative entropies measured with respect
to the bulk and boundary observables are equal. We also prove the converse:
when the relative entropies measured in an entanglement wedge and its
complement equal the relative entropies measured in their respective boundary
subregions, entanglement wedge reconstruction is possible. Along the way, we
show that the bulk and boundary modular operators act on the code subspace in
the same

## ANEC for Ronak ##

In [123]:
fast_trans_recommender(sentence_model_sbert,"1605.08072")


 1) With TFIDF score 0.2083876132965088 and GPT-2 rank 0 
 Paper title : ['Renyi entropy, stationarity, and entanglement of the conformal scalar']. 
 Abstract : ['  We extend previous work on the perturbative expansion of the Renyi entropy,\n$S_q$, around $q=1$ for a spherical entangling surface in a general CFT.\nApplied to conformal scalar fields in various spacetime dimensions, the results\nappear to conflict with the known conformal scalar Renyi entropies. On the\nother hand, the perturbative results agree with known Renyi entropies in a\nvariety of other theories, including theories of free fermions and vector\nfields and theories with Einstein gravity duals. We propose a resolution\nstemming from a careful consideration of boundary conditions near the\nentangling surface. This is equivalent to a proper treatment of\ntotal-derivative terms in the definition of the modular Hamiltonian. As a\ncorollary, we are able to resolve an outstanding puzzle in the literature\nregarding the R


 8) With TFIDF score 0.12691669166088104 and GPT-2 rank 7 
 Paper title : ['On Renyi entropy for free conformal fields: holographic and q-analog\n  recipes']. 
 Abstract : ["  We describe a holographic approach to explicitly compute the universal\nlogarithmic contributions to entanglement and Renyi entropies for free\nconformal scalar and spinor fields on even-dimensional spheres. This\nholographic derivation proceeds in two steps: first, following Casini and\nHuerta, a conformal map to thermal entropy in a hyperbolic geometry; then,\nidentification of the hyperbolic geometry with the conformal boundary of a bulk\nhyperbolic space and use of an AdS/CFT holographic formula to compute the\nresulting functional determinant. We explicitly verify the connection with the\ntype-A trace anomaly for the entanglement entropy, whereas the Renyi entropy is\ncomputed with aid of the Sommerfeld formula in order to deal with a conical\ndefect. As a by-product, we show that the log-coefficient of the


 14) With TFIDF score 0.15843743085861206 and GPT-2 rank 14 
 Paper title : ['Islands for Reflected Entropy']. 
 Abstract : ['  Recent work has demonstrated the need to include contributions from\nentanglement islands when computing the entanglement entropy in QFT states\ncoupled to regions of semiclassical gravity. We propose a new formula for the\nreflected entropy that includes additional contributions from such islands. We\nderive this formula from the gravitational path integral by finding additional\nsaddles that include generalized replica wormholes. We also demonstrate that\nour covariant formula satisfies all the inequalities required of the reflected\nentropy. We use this formula in various examples that demonstrate its relevance\nin illustrating the structure of multipartite entanglement that are invisible\nto the entropies.\n']. 
 Authors : ['Venkatesa Chandrasekaran, Masamichi Miyaji and Pratik Rath']. 
 Arxiv ID : ['2006.10754'] 
 
 

 15) With TFIDF score 0.102163568139


 23) With TFIDF score 0.1633027195930481 and GPT-2 rank 27 
 Paper title : ['Holographic Subregion Complexity from Kinematic Space']. 
 Abstract : ['  We consider the computation of volumes contained in a spatial slice of\nAdS$_3$ in terms of observables in a dual CFT. Our main tool is kinematic\nspace, defined either from the bulk perspective as the space of oriented bulk\ngeodesics, or from the CFT perspective as the space of entangling intervals. We\ngive an explicit formula for the volume of a general region in the spatial\nslice as an integral over kinematic space. For the region lying below a\ngeodesic, we show how to write this volume purely in terms of entangling\nentropies in the dual CFT. This expression is perhaps most interesting in light\nof the complexity=volume proposal, which posits that complexity of holographic\nquantum states is computed by bulk volumes. An extension of this idea proposes\nthat the holographic subregion complexity of an interval, defined as the volu


 29) With TFIDF score 0.32230639457702637 and GPT-2 rank 34 
 Paper title : ['A General Proof of the Quantum Null Energy Condition']. 
 Abstract : ['  We prove a conjectured lower bound on $\\left< T_{--}(x) \\right>_\\psi$ in any\nstate $\\psi$ of a relativistic QFT dubbed the Quantum Null Energy Condition\n(QNEC). The bound is given by the second order shape deformation, in the null\ndirection, of the geometric entanglement entropy of an entangling cut passing\nthrough $x$. Our proof involves a combination of the two independent methods\nthat were used recently to prove the weaker Averaged Null Energy Condition\n(ANEC). In particular the properties of modular Hamiltonians under shape\ndeformations for the state $\\psi$ play an important role, as do causality\nconsiderations. We study the two point function of a "probe" operator\n$\\mathcal{O}$ in the state $\\psi$ and use a lightcone limit to evaluate this\ncorrelator. Instead of causality in time we consider \\emph{causality in mod

In [124]:
fast_trans_recommender(sentence_model_gpt2,"1605.08072") 


 1) With TFIDF score 0.12385721504688263 and GPT-2 rank 0 
 Paper title : ['Edge state inner products and real-space entanglement spectrum of trial\n  quantum Hall states']. 
 Abstract : ['  We consider the trial wavefunctions for the Fractional Quantum Hall Effect\n(FQHE) that are given by conformal blocks, and construct their associated edge\nexcited states in full generality. The inner products between these edge states\nare computed in the thermodynamic limit, assuming generalized screening (i.e.\nshort-range correlations only) inside the quantum Hall droplet, and using the\nlanguage of boundary conformal field theory (boundary CFT). These inner\nproducts take universal values in this limit: they are equal to the\ncorresponding inner products in the bulk 2d chiral CFT which underlies the\ntrial wavefunction. This is a bulk/edge correspondence; it shows the equality\nbetween equal-time correlators along the edge and the correlators of the bulk\nCFT up to a Wick rotation. This appro


 6) With TFIDF score 0.20433205366134644 and GPT-2 rank 5 
 Paper title : ['A Conformal Collider for Holographic CFTs']. 
 Abstract : ['  We develop a formalism to study the implications of causality on OPE\ncoefficients in conformal field theories with large central charge and a sparse\nspectrum of higher spin operators. The formalism has the interpretation of a\nnew conformal collider-type experiment for these class of CFTs and hence it has\nthe advantage of requiring knowledge only about CFT three-point functions. This\nis accomplished by considering the holographic null energy operator which was\nintroduced in arXiv:1709.03597 as a generalization of the averaged null energy\noperator. Analyticity properties of correlators in the Regge limit imply that\nthe holographic null energy operator is a positive operator in a subspace of\nthe total CFT Hilbert space. Utilizing this positivity condition, we derive\nbounds on three-point functions $\\langle TO_1O_2\\rangle$ of the stress tens


 11) With TFIDF score 0.1876790076494217 and GPT-2 rank 10 
 Paper title : ['Positivity, entanglement entropy, and minimal surfaces']. 
 Abstract : ["  The path integral representation for the Renyi entanglement entropies of\ninteger index n implies these information measures define operator correlation\nfunctions in QFT. We analyze whether the limit $n\\rightarrow 1$, corresponding\nto the entanglement entropy, can also be represented in terms of a path\nintegral with insertions on the region's boundary, at first order in $n-1$.\nThis conjecture has been used in the literature in several occasions, and\nspecially in an attempt to prove the Ryu-Takayanagi holographic entanglement\nentropy formula. We show it leads to conditional positivity of the entropy\ncorrelation matrices, which is equivalent to an infinite series of polynomial\ninequalities for the entropies in QFT or the areas of minimal surfaces\nrepresenting the entanglement entropy in the AdS-CFT context. We check these\nineq


 17) With TFIDF score 0.19290097057819366 and GPT-2 rank 22 
 Paper title : ['Conformal perturbation theory and higher spin entanglement entropy on\n  the torus']. 
 Abstract : ['  We study the free fermion theory in 1+1 dimensions deformed by chemical\npotentials for holomorphic, conserved currents at finite temperature and on a\nspatial circle. For a spin-three chemical potential \\mu, the deformation is\nrelated at high temperatures to a higher spin black hole in hs[0] theory on\nAdS_3 spacetime. We calculate the order \\mu^2 corrections to the single\ninterval Renyi and entanglement entropies on the torus using the bosonized\nformulation. A consistent result, satisfying all checks, emerges upon carefully\naccounting for both perturbative and winding mode contributions in the\nbosonized language. The order \\mu^2 corrections involve integrals that are\nfinite but potentially sensitive to contact term singularities. We propose and\napply a prescription for defining such integrals wh


 22) With TFIDF score 0.2039056122303009 and GPT-2 rank 36 
 Paper title : ['A perturbative study on the analytic continuation for generalized\n  gravitational entropy']. 
 Abstract : ['  We study the analytic continuation used by Lewkowycz and Maldacena to prove\nthe Ryu-Takayanagi formula for entanglement entropy, which is the holographic\ndual of the trace of the $\\beta$-power of the time evolution operator when\n$\\beta\\in \\mathbb{R}$. This will be done perturbatively by using a weakly time\ndependent Hamiltonian, corresponding to a small shift of the dual static\nbackground. Depending on the periodicity we impose on the gravitational\nsolution, we consider two different possibilities and compare the associated\nentropies with the results obtained through a minimal area computation. To our\nsurprise we discover that, at first order, both choices correctly reproduce the\nassociated entanglement entropy. Furthermore we find unexpected divergent\ncontributions that we have to disc


 28) With TFIDF score 0.14780479669570923 and GPT-2 rank 53 
 Paper title : ['Symmetries in quantum field theory and quantum gravity']. 
 Abstract : ['  In this paper we use the AdS/CFT correspondence to refine and then establish\na set of old conjectures about symmetries in quantum gravity. We first show\nthat any global symmetry, discrete or continuous, in a bulk quantum gravity\ntheory with a CFT dual would lead to an inconsistency in that CFT, and thus\nthat there are no bulk global symmetries in AdS/CFT. We then argue that any\n"long-range" bulk gauge symmetry leads to a global symmetry in the boundary\nCFT, whose consistency requires the existence of bulk dynamical objects which\ntransform in all finite-dimensional irreducible representations of the bulk\ngauge group. We mostly assume that all internal symmetry groups are compact,\nbut we also give a general condition on CFTs, which we expect to be true quite\nbroadly, which implies this. We extend all of these results to the ca

In [125]:
recommend_doc2vec(model_big_bow300_5_10,"1605.08072",30) 

1) Modular Hamiltonians for Euclidean Path Integral States.
 Abstracts :   We study half-space/Rindler modular Hamiltonians for excited states created
by turning on sources for local operators in the Euclidean path integral in
relativistic quantum field theories. We derive a simple, manifestly Lorentzian
formula for the modular Hamiltonian to all orders in perturbation theory in the
sources. We apply this formula to the case of shape-deformed half spaces in the
vacuum state, and obtain the corresponding modular Hamiltonian to all orders in
the shape deformation in terms of products of half-sided null energy operators,
i.e., stress tensor components integrated along the future and past Rindler
horizons. In the special case where the shape deformation is purely null, our
perturbation series can be resummed, and agrees precisely with the known
formula for vacuum modular Hamiltonians for null cuts of the Rindler horizon.
Finally, we study some universal properties of modular flow (correspo

## Raghu for Raghu ##

In [122]:
fast_trans_recommender(sentence_model_gpt2,"1908.10996") 


 1) With TFIDF score 0.563462495803833 and GPT-2 rank 0 
 Paper title : ['Geometric secret sharing in a model of Hawking radiation']. 
 Abstract : ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new ru


 6) With TFIDF score 0.3986660838127136 and GPT-2 rank 5 
 Paper title : ['Entanglement Wedge Reconstruction and the Information Paradox']. 
 Abstract : ['  When absorbing boundary conditions are used to evaporate a black hole in\nAdS/CFT, we show that there is a phase transition in the location of the\nquantum Ryu-Takayanagi surface, at precisely the Page time. The new RT surface\nlies slightly inside the event horizon, at an infalling time approximately the\nscrambling time $\\beta/2\\pi \\log S_{BH}$ into the past. We can immediately\nderive the Page curve, using the Ryu-Takayanagi formula, and the\nHayden-Preskill decoding criterion, using entanglement wedge reconstruction.\nBecause part of the interior is now encoded in the early Hawking radiation, the\ndecreasing entanglement entropy of the black hole is exactly consistent with\nthe semiclassical bulk entanglement of the late-time Hawking modes, despite the\nabsence of a firewall.\n  By studying the entanglement wedge of highly 


 11) With TFIDF score 0.272577702999115 and GPT-2 rank 10 
 Paper title : ['Black hole entropy and the renormalization group']. 
 Abstract : ['  Four decades after its first postulation by Bekenstein, black hole entropy\nremains mysterious. It has long been suggested that the entanglement entropy of\nquantum fields on the black hole gravitational background should represent at\nleast an important contribution to the total Bekenstein-Hawking entropy, and\nthat the divergences in the entanglement entropy should be absorbed in the\nrenormalization of the gravitational couplings. In this talk, we describe how\nan improved understanding of black hole entropy is obtained by combining these\nnotions with the renormalization group. By introducing an RG flow scale, we\ninvestigate whether the total entropy of the black hole can be partitioned in a\n"gravitational" part related to the flowing gravitational action, and a\n"quantum" part related to the unintegrated degrees of freedom. We describe


 17) With TFIDF score 0.221995547413826 and GPT-2 rank 16 
 Paper title : ['Holographic Evolution of Entanglement Entropy']. 
 Abstract : ['  We study the evolution of entanglement entropy in a 2-dimensional\nequilibration process that has a holographic description in terms of a Vaidya\ngeometry. It models a unitary evolution in which the field theory starts in a\npure state, its vacuum, and undergoes a perturbation that brings it far from\nequilibrium. The entanglement entropy in this set up provides a measurement of\nthe quantum entanglement in the system. Using holographic techniques we recover\nthe same result obtained before from the study of processes triggered by a\nsudden change in a parameter of the hamiltonian, known as quantum quenches.\nNamely, entanglement in 2-dimensional conformal field theories propagates with\nvelocity v^2=1. Both in quantum quenches and in the Vaidya model equilibration\nis only achieved at the local level. Remarkably, the holographic derivation of\n


 23) With TFIDF score 0.3419891595840454 and GPT-2 rank 22 
 Paper title : ['Islands in de Sitter space']. 
 Abstract : ['  We consider black holes in 2d de Sitter JT gravity coupled to a CFT, and\nentangled with matter in a disjoint non-gravitating universe. Tracing out the\nentangling matter leaves the CFT in a density matrix whose stress tensor\nbackreacts on the de Sitter geometry, lengthening the wormhole behind the black\nhole horizon. Naively, the entropy of the entangling matter increases without\nbound as the strength of the entanglement increases, but the monogamy property\npredicts that this growth must level off. We compute the entropy via the\nreplica trick, including wormholes between the replica copies of the de Sitter\ngeometry, and find a competition between conventional field theory entanglement\nentropy and the surface area of extremal "islands" in the de Sitter geometry.\nThe black hole and cosmological horizons both play a role in generating such\nislands in the b


 29) With TFIDF score 0.2539184093475342 and GPT-2 rank 28 
 Paper title : ['Entanglement dynamics after quantum quenches in generic integrable\n  systems']. 
 Abstract : ['  The time evolution of the entanglement entropy in non-equilibrium quantum\nsystems provides crucial information about the structure of the time-dependent\nstate. For quantum quench protocols, by combining a quasiparticle picture for\nthe entanglement spreading with the exact knowledge of the stationary state\nprovided by Bethe ansatz, it is possible to obtain an exact and analytic\ndescription of the evolution of the entanglement entropy. Here we discuss the\napplication of these ideas to several integrable models. First we show that for\nnon-interacting systems, both bosonic and fermionic, the exact time-dependence\nof the entanglement entropy can be derived by elementary techniques and without\nsolving the dynamics. We then provide exact results for interacting spin chains\nthat are carefully tested against num

In [96]:
fast_trans_recommender(sentence_model_sbert,"1908.10996") #Raghu


 1) With TFIDF score 0.563462495803833 and GPT-2 rank 0 
 Paper title : ['Geometric secret sharing in a model of Hawking radiation']. 
 Abstract : ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new ru


 8) With TFIDF score 0.20133934915065765 and GPT-2 rank 7 
 Paper title : ['No Holography for Eternal AdS Black Holes']. 
 Abstract : ['  It is generally believed that the eternal AdS black hole is dual to two\nconformal field theories with compact spatial sections that are together in a\nthermofield double state. We argue that this proposal is incorrect, and by\nextension so are the "entanglement=geometry" proposal of Van Raamsdonk and\n"ER=EPR" proposal of Maldacena and Susskind. We show that in the bulk there is\nan interaction needed between the two halves of the Hilbert space for\nconnectivity across the horizon; however, there is no such interaction between\nthe CFTs. This rules out the possibility of the dual to the CFTs being the\neternal AdS black hole. We argue the correct dual "geometries" resemble the\nexterior of the black hole outside the stretched horizon but cap off before the\nglobal horizon. This disallows the possibility of a shared future (and past)\nwedge where Al


 14) With TFIDF score 0.12314967066049576 and GPT-2 rank 14 
 Paper title : ['Holographic screens in ultraviolet self-complete quantum gravity']. 
 Abstract : ['  In this paper we study the geometry and the thermodynamics of a holographic\nscreen in the framework of the ultraviolet self-complete quantum gravity. To\nachieve this goal we construct a new static, neutral, non-rotating black hole\nmetric, whose outer (event) horizon coincides with the surface of the screen.\nThe space-time admits an extremal configuration corresponding to the minimal\nholographic screen and having both mass and radius equalling the Planck units.\nWe identify this object as the space-time fundamental building block, whose\ninterior is physically unaccessible and cannot be probed even during the\nHawking evaporation terminal phase. In agreement with the holographic\nprinciple, relevant processes take place on the screen surface. The area\nquantization leads to a discrete mass spectrum. An analysis of the en


 22) With TFIDF score 0.2728791832923889 and GPT-2 rank 23 
 Paper title : ['Islands outside the horizon']. 
 Abstract : ['  We consider an AdS$_2$ black hole in equilibrium with a bath, which we take\nto have a dual description as (0+1)-dimensional quantum mechanical system\ncoupled to a (1+1)-dimensional field theory serving as the bath. We compute the\nentropies of both the quantum mechanical degrees of freedom and of the bath\nseparately, while allowing contributions from entanglement wedge "islands". We\nfind situations where the island extends {\\it outside} the black hole horizon.\nThis suggests possible causality paradoxes which we show are avoided because of\nthe quantum focusing conjecture. Finally, we formulate a version of the\ninformation paradox for a black hole in contact with a bath in the\nHartle-Hawking state, and demonstrate the role of islands in resolving this\nparadox.\n']. 
 Authors : ['Ahmed Almheiri, Raghu Mahajan, Juan Maldacena']. 
 Arxiv ID : ['1910.11077']


 29) With TFIDF score 0.12483520060777664 and GPT-2 rank 30 
 Paper title : ['Liouville Theory on Horizons: Towards a Quantum Theory of Black Holes']. 
 Abstract : ['  We show that any nonextremal black hole can be described by a Liouville\ntheory that lives on its very near horizon region. In classical Liouville\ntheory, the black hole corresponds to a field configuration that reproduces the\nRindler metric. In quantum Liouville theory, the black hole state in the CFT is\ncreated by the puncture operator that gives rise to a conical singularity with\na deficit angle of $2 \\pi$. This state is "heavy" with a nonnormalizable wave\nfunction in the minisuperspace approximation. Black hole entropy counts the\nnumber of ways the "heavy" black hole state can be obtained by acting with a\nproduct of "light, quantum" operators in the CFT. Black hole hair is described\nby the "background charge" in the CFT but its physical nature is unclear.\n']. 
 Authors : ['Edi Halyo']. 
 Arxiv ID : ['1606.

In [107]:
recommend_doc2vec(model_big_bow300_5_10,"1908.10996",30) #Raghu

1) Geometric secret sharing in a model of Hawking radiation.
 Abstracts :   We consider a black hole in three dimensional AdS space entangled with an
auxiliary radiation system. We model the microstates of the black hole in terms
of a field theory living on an end of the world brane behind the horizon, and
allow this field theory to itself have a holographic dual geometry. This
geometry is also a black hole since entanglement of the microstates with the
radiation leaves them in a mixed state. This "inception black hole" can be
purified by entanglement through a wormhole with an auxiliary system which is
naturally identified with the external radiation, giving a realization of the
ER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi
(RT) formula, in which extremal surfaces computing entanglement entropy are
allowed to pass through the brane into its dual geometry. This new rule
reproduces the Page curve for evaporating black holes, consistently with the
recent

## Causality constraints for Raghu ##

In [126]:
fast_trans_recommender(sentence_model_sbert,"1509.00014")


 1) With TFIDF score 0.2161896973848343 and GPT-2 rank 0 
 Paper title : ['Constraining Conformal Theories in Large Dimensions']. 
 Abstract : ['  In this paper, we analyze the constraints imposed by unitarity and crossing\nsymmetry on conformal theories in large dimensions. In particular, we show that\nin a unitary conformal theory in large dimension $D$, the four-point function\nof identical scalar operators $\\phi$ with scaling dimension $\\Delta_\\phi$ such\nthat $\\Delta_\\phi/D<3/4$, is necessarily that of the generalized free field\ntheory. This result follows only from crossing symmetry and unitarity. In\nparticular, we do not impose the existence of a conserved spin two operator\n(stress tensor). We also present an argument to extend the applicability of\nthis result to a larger range of conformal dimensions, namely to\n$\\Delta_\\phi/D<1$. This extension requires some reasonable assumptions about\nthe spectrum of light operators. Together, these results suggest that if there


 8) With TFIDF score 0.12306341528892517 and GPT-2 rank 8 
 Paper title : ['Revisiting the dilatation operator of the Wilson-Fisher fixed point']. 
 Abstract : ['  We revisit the order $\\varepsilon$ dilatation operator of the Wilson-Fisher\nfixed point obtained by Kehrein, Pismak, and Wegner in light of recent results\nin conformal field theory. Our approach is algebraic and based only on symmetry\nprinciples. The starting point of our analysis is that the first correction to\nthe dilatation operator is a conformal invariant, which implies that its form\nis fixed up to an infinite set of coefficients associated with the scaling\ndimensions of higher-spin currents. These coefficients can be fixed using\nwell-known perturbative results, however, they were recently re-obtained using\nCFT arguments without relying on perturbation theory. Our analysis then implies\nthat all order-$\\varepsilon$ scaling dimensions of the Wilson-Fisher fixed\npoint can be fixed by symmetry.\n']. 
 Authors :


 15) With TFIDF score 0.14086538553237915 and GPT-2 rank 16 
 Paper title : ['Pure contact term correlators in CFT']. 
 Abstract : ['  We discuss the case of correlators in CFT made of pure contact terms, without\na corresponding bare part. We show two examples. The first is provided by the\nconformal limits of a free massive fermion theory in 3d. We show that the\n(conserved) current correlators are in one-to-one correspondence with the terms\nof the 3d gauge CS action. The second is the Pontryagin trace anomaly. The\ncorresponding 3-point correlator is nonvanishing even though the corresponding\nuntraced correlator vanishes.\n']. 
 Authors : ['Loriano Bonora and Bruno Lima de Souza']. 
 Arxiv ID : ['1511.06635'] 
 
 

 16) With TFIDF score 0.18460369110107422 and GPT-2 rank 18 
 Paper title : ['Analyticity in Spin in Conformal Theories']. 
 Abstract : ['  Conformal theory correlators are characterized by the spectrum and three-\npoint functions of local operators. We present a formu


 22) With TFIDF score 0.15697064995765686 and GPT-2 rank 26 
 Paper title : ['Conformal Bootstrap With Slightly Broken Higher Spin Symmetry']. 
 Abstract : ['  We consider conformal field theories with slightly broken higher spin\nsymmetry in arbitrary spacetime dimensions. We analyze the crossing equation in\nthe double light-cone limit and solve for the anomalous dimensions of higher\nspin currents $\\gamma_s$ with large spin $s$. The result depends on the\nsymmetries and the spectrum of the unperturbed conformal field theory. We\nreproduce all known results and make further predictions. In particular we make\na prediction for the anomalous dimensions of higher spin currents in the 3d\nIsing model.\n']. 
 Authors : ['Luis F. Alday and Alexander Zhiboedov']. 
 Arxiv ID : ['1506.04659'] 
 
 

 23) With TFIDF score 0.12288786470890045 and GPT-2 rank 27 
 Paper title : ['Beyond $a=c$: Gravitational Couplings to Matter and the Stress Tensor\n  OPE']. 
 Abstract : ['  We derive constraint


 29) With TFIDF score 0.22185765206813812 and GPT-2 rank 37 
 Paper title : ['Conformal blocks for highly disparate scaling dimensions']. 
 Abstract : ['  We show that conformal blocks simplify greatly when there is a large\ndifference between two of the scaling dimensions for external operators. In\nparticular the spacetime dimension only appears in an overall constant which we\ndetermine via recurrence relations. Connections to the conformal bootstrap\nprogram and the AdS / CFT correspondence are also discussed.\n']. 
 Authors : ['Connor Behan']. 
 Arxiv ID : ['1402.5698'] 
 
 

 30) With TFIDF score 0.1845967173576355 and GPT-2 rank 38 
 Paper title : ['Schwinger-Keldysh Propagators from AdS/CFT Correspondence']. 
 Abstract : ["  We demonstrate how to compute real-time Green's functions for a class of\nfinite temperature field theories from their AdS gravity duals. In particular,\nwe reproduce the two-by-two Schwinger-Keldysh matrix propagator from a gravity\ncalculation. Our metho

In [129]:
fast_trans_recommender(sentence_model_gpt2,"1509.00014")


 1) With TFIDF score 0.212433859705925 and GPT-2 rank 0 
 Paper title : ['Averaged Null Energy Condition from Causality']. 
 Abstract : ['  Unitary, Lorentz-invariant quantum field theories in flat spacetime obey\nmicrocausality: commutators vanish at spacelike separation. For interacting\ntheories in more than two dimensions, we show that this implies that the\naveraged null energy, $\\int du T_{uu}$, must be positive. This non-local\noperator appears in the operator product expansion of local operators in the\nlightcone limit, and therefore contributes to $n$-point functions. We derive a\nsum rule that isolates this contribution and is manifestly positive. The\nargument also applies to certain higher spin operators other than the stress\ntensor, generating an infinite family of new constraints of the form $\\int du\nX_{uuu\\cdots u} \\geq 0$. These lead to new inequalities for the coupling\nconstants of spinning operators in conformal field theory, which include as\nspecial cases (b


 6) With TFIDF score 0.14583468437194824 and GPT-2 rank 8 
 Paper title : ['Extremal Correlators and Random Matrix Theory']. 
 Abstract : ['  We study the correlation functions of Coulomb branch operators of\nfour-dimensional $\\mathcal{N} = 2$ Superconformal Field Theories (SCFTs). We\nfocus on rank-one theories, such as the SU(2) gauge theory with four\nfundamental hypermultiplets. "Extremal" correlation functions, involving\nexactly one anti-chiral operator, are perhaps the simplest nontrivial\ncorrelation functions in four-dimensional Quantum Field Theory. We show that\nthe large charge limit of extremal correlators is captured by a "dual"\ndescription which is a chiral random matrix model of the Wishart-Laguerre type.\nThis gives an analytic handle on the physics in some particular excited states.\nIn the limit of large random matrices we find the physics of a non-relativistic\naxion-dilaton effective theory. The random matrix model also admits a \'t Hooft\nexpansion in which the


 11) With TFIDF score 0.1877417266368866 and GPT-2 rank 16 
 Paper title : ['Projectors, Shadows, and Conformal Blocks']. 
 Abstract : ['  We introduce a method for computing conformal blocks of operators in\narbitrary Lorentz representations in any spacetime dimension, making it\npossible to apply bootstrap techniques to operators with spin. The key idea is\nto implement the "shadow formalism" of Ferrara, Gatto, Grillo, and Parisi in a\nsetting where conformal invariance is manifest. Conformal blocks in\n$d$-dimensions can be expressed as integrals over the projective null-cone in\nthe "embedding space" $\\mathbb{R}^{d+1,1}$. Taking care with their analytic\nstructure, these integrals can be evaluated in great generality, reducing the\ncomputation of conformal blocks to a bookkeeping exercise. To facilitate\ncalculations in four-dimensional CFTs, we introduce techniques for writing down\nconformally-invariant correlators using auxiliary twistor variables, and\ndemonstrate their use i


 16) With TFIDF score 0.1267593950033188 and GPT-2 rank 23 
 Paper title : ['Comments on Higher-Spin Fields in Nontrivial Backgrounds']. 
 Abstract : ['  We consider the free propagation of totally symmetric massive bosonic fields\nin nontrivial backgrounds. The mutual compatibility of the dynamical equations\nand constraints in flat space amounts to the existence of an Abelian algebra\nformed by the d\'Alembertian, divergence and trace operators. The latter, along\nwith the symmetrized gradient, symmetrized metric and spin operators, actually\ngenerate a bigger non-Abelian algebra, which we refer to as the "consistency"\nalgebra. We argue that in nontrivial backgrounds, it is some deformed version\nof this algebra that governs the consistency of the system. This can be\nmotivated, for example, from the theory of charged open strings in a background\ngauge field, where the Virasoro algebra ensures consistent propagation. For a\ngravitational background, we outline a systematic procedu


 21) With TFIDF score 0.1104312390089035 and GPT-2 rank 34 
 Paper title : ['Linear derivative Cartan formulation of General Relativity']. 
 Abstract : ["  Beside diffeomorphism invariance also manifest SO(3,1) local Lorentz\ninvariance is implemented in a formulation of Einstein Gravity (with or without\ncosmological term) in terms of initially completely independent vielbein and\nspin connection variables and auxiliary two-form fields. In the systematic\nstudy of all possible embeddings of Einstein gravity into that formulation with\nauxiliary fields, the introduction of a ``bi-complex'' algebra possesses\ncrucial technical advantages. Certain components of the new two-form fields\ndirectly provide canonical momenta for spatial components of all Cartan\nvariables, whereas the remaining ones act as Lagrange multipliers for a large\nnumber of constraints, some of which have been proposed already in different,\nless radical approaches. The time-like components of the Cartan variables p


 26) With TFIDF score 0.14097420871257782 and GPT-2 rank 43 
 Paper title : ['Higher Spin Entanglement and W_N Conformal Blocks']. 
 Abstract : ["  Two-dimensional conformal field theories with extended $\\cal{W}$-symmetry\nalgebras have dual descriptions in terms of weakly coupled higher spin gravity\nin AdS$_3\\,$ at large central charge. Observables that can be computed and\ncompared in the two descriptions include R\\'enyi and entanglement entropies,\nand correlation functions of local operators. We develop techniques for\ncomputing these, in a manner that sheds light on when and why one can expect\nagreement between such quantities on each side of the duality. We set up the\ncomputation of excited state R\\'enyi entropies in the bulk in terms of\nChern-Simons connections, and show how this directly parallels the CFT\ncomputation of correlation functions. More generally, we consider the vacuum\nconformal block for general operators with $\\Delta \\sim c\\,$. When two of the\nopera

In [127]:
recommend_doc2vec(model_big_bow300_5_10,"1509.00014",30) #Doc2Vec

1) A Generalized Nachtmann Theorem in CFT.
 Abstracts :   Correlators of unitary quantum field theories in Lorentzian signature obey
certain analyticity and positivity properties. For interacting unitary CFTs in
more than two dimensions, we show that these properties impose general
constraints on families of minimal twist operators that appear in the OPEs of
primary operators. In particular, we rederive and extend the convexity theorem
which states that for the family of minimal twist operators with even spins
appearing in the reflection-symmetric OPE of any scalar primary, twist must be
a monotonically increasing convex function of the spin. Our argument is
completely non-perturbative and it also applies to the OPE of nonidentical
scalar primaries in unitary CFTs, constraining the twist of spinning operators
appearing in the OPE. Finally, we argue that the same methods also impose
constraints on the Regge behavior of certain CFT correlators.
  Authors : Sandipan Kundu (0.4546738266944

# Jared for Milind #

In [80]:
fast_trans_recommender(sentence_model_gpt2,"1212.3616")


 1) With TFIDF score 0.4061584770679474 and GPT-2 rank 0 
 Paper title : ['Bounding scalar operator dimensions in 4D CFT']. 
 Abstract : ['  In an arbitrary unitary 4D CFT we consider a scalar operator \\phi, and the\noperator \\phi^2 defined as the lowest dimension scalar which appears in the OPE\n\\phi\\times\\phi with a nonzero coefficient. Using general considerations of OPE,\nconformal block decomposition, and crossing symmetry, we derive a\ntheory-independent inequality [\\phi^2] \\leq f([\\phi]) for the dimensions of\nthese two operators. The function f(d) entering this bound is computed\nnumerically. For d->1 we have f(d)=2+O(\\sqrt{d-1}), which shows that the free\ntheory limit is approached continuously. We perform some checks of our bound.\nWe find that the bound is satisfied by all weakly coupled 4D conformal fixed\npoints that we are able to construct. The Wilson-Fischer fixed points violate\nthe bound by a constant O(1) factor, which must be due to the subtleties of\next


 7) With TFIDF score 0.19187623262405396 and GPT-2 rank 6 
 Paper title : ['AdS Description of Induced Higher-Spin Gauge Theory']. 
 Abstract : ["  We study deformations of three-dimensional large N CFTs by double-trace\noperators constructed from spin s single-trace operators of dimension \\Delta.\nThese theories possess UV fixed points, and we calculate the change of the\n3-sphere free energy \\delta F= F_{UV}- F_{IR}. To describe the UV fixed point\nusing the dual AdS_4 space we modify the boundary conditions on the spin s\nfield in the bulk; this approach produces \\delta F in agreement with the field\ntheory calculations. If the spin s operator is a conserved current, then the\nfixed point is described by an induced parity invariant conformal spin s gauge\ntheory. The low spin examples are QED_3 (s=1) and the 3-d induced conformal\ngravity (s=2). When the original CFT is that of N conformal complex scalar or\nfermion fields, the U(N) singlet sector of the induced 3-d gauge theory


 12) With TFIDF score 0.1640923172235489 and GPT-2 rank 12 
 Paper title : ['Structure of large spin expansion of anomalous dimensions at strong\n  coupling']. 
 Abstract : ['  The anomalous dimensions of planar N=4 SYM theory operators like tr(Phi D^S\nPhi) expanded in large spin S have the asymptotics \\gamma= f ln S + f_c + 1/S\n(f_11 ln S + f_10) + ..., where f (the universal scaling function or cusp\nanomaly), f_c and f_mn are given by power series in the `t Hooft coupling\n\\lambda. The subleading coefficients appear to be related by the so called\nfunctional relation and parity invariance (or reciprocity) property of the\nfunction expressing \\gamma in terms of the conformal spin of the collinear\ngroup. Here we study the structure of such large spin expansion at strong\ncoupling via AdS/CFT, i.e. by using the dual description in terms of folded\nspinning string in AdS_5. The large spin expansion of the classical string\nenergy happens to have the same structure as that of \\ga


 18) With TFIDF score 0.15435536205768585 and GPT-2 rank 29 
 Paper title : ['Scalar fields and three-point functions in D=3 higher spin gravity']. 
 Abstract : ["  We compute boundary three-point functions involving two scalars and a gauge\nfield of arbitrary spin in the AdS vacuum of Vasiliev's higher spin gravity,\nfor any deformation parameter \\lambda. In the process, we develop tools for\nextracting scalar field equations in arbitrary higher spin backgrounds. We work\nin the context of hs[\\lambda] + hs[\\lambda] Chern-Simons theory coupled to\nscalar fields, and make efficient use of the associative lone-star product\nunderlying the hs[\\lambda] algebra. Our results for the correlators precisely\nmatch expectations from CFT; in particular they match those of any CFT with\nW_{\\infinity}[\\lambda] symmetry at large central charge, and with primary\noperators dual to the scalar fields. As this is expected to include the `t\nHooft limit of the W_N minimal model CFT, our results se


 24) With TFIDF score 0.13235782086849213 and GPT-2 rank 55 
 Paper title : ['Partition Functions and Casimir Energies in Higher Spin AdS_{d+1}/CFT_d']. 
 Abstract : ['  Recently, the one-loop free energy of higher spin (HS) theories in Euclidean\nAdS_{d+1} was calculated and matched with the order N^0 term in the free energy\nof the large N "vectorial" scalar CFT on the S^d boundary. Here we extend this\nmatching to the boundary theory defined on S^1 x S^{d-1}, where the length of\nS^1 may be interpreted as the inverse temperature. It has been shown that the\nlarge N limit of the partition function on S^1 x S^2 in the U(N) singlet sector\nof the CFT of N free complex scalars matches the one-loop thermal partition\nfunction of the Vasiliev theory in AdS_4, while in the O(N) singlet sector of\nthe CFT of N real scalars it matches the minimal theory containing even spins\nonly. We extend this matching to all dimensions d. We also calculate partition\nfunctions for the singlet sectors of


 29) With TFIDF score 0.13434377312660217 and GPT-2 rank 71 
 Paper title : ['The leading trajectory in the 2+1D Ising CFT']. 
 Abstract : ['  We study the scattering of lumps in the 2+1-dimensional Ising CFT,\nindirectly, by analytically continuing its spectrum using the Lorentzian\ninversion formula. We find evidence that the intercept of the model is below\nunity: $j_*\\approx 0.8$, indicating that scattering is asymptotically\ntransparent corresponding to a negative Lyapunov exponent. We use as input the\nprecise spectrum obtained from the numerical conformal bootstrap. We show that\nthe truncated spectrum allows the inversion formula to reproduce the properties\nof the spin-two stress tensor to $10^{-4}$ accuracy and we address the question\nof whether the spin-0 operators of the model lie on Regge trajectories. This\nhypothesis is further supported by analytics in the large-N O(N) model.\nFinally, we show that anomalous dimensions of heavy operators decrease with\nenergy at a ra

In [33]:
recommend("1212.3616",30) #TF-IDF

1) Implications of the AdS/CFT Correspondence on Spacetime and Worldsheet
  OPE Coefficients.
 Abstracts :   We explore the connection between the operator product expansion (OPE) in the
boundary and worldsheet conformal field theories in the context of
AdS$_{d+1}$/CFT$_d$ correspondence. Considering single trace scalar operators
in the boundary theory and using the saddle point analysis of the worldsheet
OPE [1], we derive an explicit relation between OPE coefficients in the
boundary and worldsheet theories for the contribution of single trace spin
$\ell$ operators to the OPE. We also consider external vector operators and
obtain the relation between OPE coefficients for the exchange of single trace
scalar operators in the OPE. We revisit the relationship between the bulk cubic
couplings in the Supergravity approximation and the OPE coefficients in the
dual boundary theory. Our results match with the known examples from the case
of AdS$_3$/CFT$_2$. For the operators whose two and thre

# DSD for Milind #

In [84]:
fast_trans_recommender(sentence_model_sbert,"1805.00098")


 1) With TFIDF score 0.48161259293556213 and GPT-2 rank 1 
 Paper title : ['The light-ray OPE and conformal colliders']. 
 Abstract : ['  We derive a nonperturbative, convergent operator product expansion (OPE) for\nnull-integrated operators on the same null plane in a CFT. The objects\nappearing in the expansion are light-ray operators, whose matrix elements can\nbe computed by the generalized Lorentzian inversion formula. For example, a\nproduct of average null energy (ANEC) operators has an expansion in the\nlight-ray operators that appear in the stress-tensor OPE. An important\napplication is to collider event shapes. The light-ray OPE gives a\nnonperturbative expansion for event shapes in special functions that we call\ncelestial blocks. As an example, we apply the celestial block expansion to\nenergy-energy correlators in N=4 Super Yang-Mills theory. Using known OPE data,\nwe find perfect agreement with previous results both at weak and strong\ncoupling, and make new predictions


 9) With TFIDF score 0.35733529925346375 and GPT-2 rank 15 
 Paper title : ['A new representation for two- and three-point correlators of operators\n  from sl(2) sector']. 
 Abstract : ['  We construct a new representation for two- and three-point correlators of\noperators from sl(2) sector of planar N = 4 SYM. The spin and twist of\noperators are arbitrary. We start with the correlation function of light-ray\noperators and carry out a projection to particular local operators using the\nmethod of Separated Variables. With the same calculation we obtain polynomials\nwhich are dual to wave functions of sl(2,R) spin-chain.\n']. 
 Authors : ['Evgeny Sobko (Ecole Normale Superieure)']. 
 Arxiv ID : ['1311.6957'] 
 
 

 10) With TFIDF score 0.14711976051330566 and GPT-2 rank 17 
 Paper title : ['Anomalous Dimensions from Crossing Kernels']. 
 Abstract : ['  In this note we consider the problem of extracting the corrections to CFT\ndata induced by the exchange of a primary operator and its d


 16) With TFIDF score 0.21288949251174927 and GPT-2 rank 32 
 Paper title : ['Lorentzian CFT 3-point functions in momentum space']. 
 Abstract : ['  In a conformal field theory, two and three-point functions of scalar\noperators and conserved currents are completely determined, up to constants, by\nconformal invariance. The expressions for these correlators in Euclidean\nsignature are long known in position space, and were fully worked out in recent\nyears in momentum space. In Lorentzian signature, the position-space\ncorrelators simply follow from the Euclidean ones by means of the i-epsilon\nprescription. In this paper, we compute the Lorentzian correlators in momentum\nspace and in arbitrary dimensions for three scalar operators by means of a\nformal Wick rotation. We explain how tensorial three-point correlators can be\nobtained and, in particular, compute the correlator with two identical scalars\nand one energy-momentum tensor. As an application, we show that expectation\nvalue


 25) With TFIDF score 0.19300325214862823 and GPT-2 rank 57 
 Paper title : ['The gravity dual of Lorentzian OPE blocks']. 
 Abstract : ['  We consider the operator product expansion (OPE) structure of scalar primary\noperators in a generic Lorentzian CFT and its dual description in a\ngravitational theory with one extra dimension. The OPE can be decomposed into\ncertain bi-local operators transforming as the irreducible representations\nunder conformal group, called the OPE blocks. We show the OPE block is given by\nintegrating a higher spin field along a geodesic in the Lorentzian AdS\nspace-time when the two operators are space-like separated. When the two\noperators are time-like separated however, we find the OPE block has a peculiar\nrepresentation where the dual gravitational theory is not defined on the AdS\nspace-time but on a hyperboloid with an additional time coordinate and\nMinkowski space-time on its boundary. This differs from the surface Witten\ndiagram proposal for th

In [34]:
recommend("1805.00098",30)

1) Light-ray Operators and the BMS Algebra.
 Abstracts :   We study light-ray operators constructed from the energy-momentum tensor in
$d$-dimensional Lorentzian conformal field theory. These include in particular
the average null energy operator. The commutators of parallel light-ray
operators on a codimension one light-sheet form an infinite-dimensional
algebra. We determine this light-ray algebra and find that the $d$-dimensional
(generalized) BMS algebra, including both the supertranslation and the
superrotation, is a subalgebra. We verify this algebra in correlation functions
of free scalar field theory. We also determine the infinite-dimensional algebra
of light-ray operators built from non-abelian spin-one conserved currents.
  Authors : Clay Cordova and Shu-Heng Shao (0.7257094). 
 Arxiv: 1810.05706 
 

2) Transverse spin in the light-ray OPE.
 Abstracts :   We study a product of null-integrated local operators $\mathcal{O}_1$ and
$\mathcal{O}_2$ on the same null plane in a CFT